<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/789101112_50000_shuffled_ep8_alpha10_gamma15_micro_samples_exact_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time

# Wait until /content is writable — sign that Colab is ready
print("⏳ Waiting for runtime to be ready...", end="", flush=True)

while not os.path.exists("/content") or not os.access("/content", os.W_OK):
    time.sleep(0.5)
    print(".", end="", flush=True)

print("\n✅ Runtime is ready. Continuing...")


⏳ Waiting for runtime to be ready...
✅ Runtime is ready. Continuing...


### Datasets and Transformers
- 2 Hugging Face libraries
- well integrated with each other
- Datasets also well integrated with PyTorch and TensorFlow libraries

In [ ]:
!pip install -qU transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 44.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.4
    Uninstalling huggingface-hub-0.33.4:
      Successfully uninstalled huggingface-hub-0.33.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


### Update the Hugging Face library 'datasets'; ignore the following message
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia...
gcsfs 2025.3.2 requires fsspec

- Upgrade: datasets 2.14.4 => datasets-3.6.0 because 2.14.4 has bugs with Path
- I don't use gcsfs


In [ ]:
!pip install -qU datasets


In [ ]:
!pip install -qU huggingface_hub

In [ ]:
!pip install -qU accelerate

In [ ]:
!pip install -q scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00


In [ ]:
import datasets
import importlib
import json
import logging
import numpy as np
import os
import pandas as pd
import pickle
import shutil
import sys
import time
import torch
import torch.nn.functional as F
import wandb
import zipfile

from contextlib                    import suppress
from datasets                      import Dataset, DatasetDict, load_from_disk, load_dataset
from datetime                      import datetime, timezone
from google.colab                  import auth, drive, files, runtime, userdata
from huggingface_hub               import create_branch, create_repo, HfApi, login, upload_file, hf_hub_download, whoami
from huggingface_hub.errors        import RepositoryNotFoundError
from logging.handlers              import RotatingFileHandler
from pathlib                       import Path
from scipy.optimize                import differential_evolution
from sklearn.metrics               import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score, accuracy_score, hamming_loss, classification_report,  precision_recall_fscore_support
from torch.optim                   import AdamW
from torch.utils.data              import DataLoader
from tqdm.auto                     import tqdm
from transformers                  import LongformerTokenizerFast, LongformerForSequenceClassification
from transformers.trainer_utils    import EvalPrediction
from transformers.trainer          import Trainer
from transformers.training_args    import TrainingArguments
from transformers.trainer_callback import TrainerCallback
#from transformers                  import logging as transformers_logging
from torch.nn                      import BCEWithLogitsLoss, Module
from zoneinfo                      import ZoneInfo


### Configuration
1. UPDATE THE ipynb FILENAME
2. GIVE ACCESS TO HF
3. UPDATE THE HF HUB REPO_ID
4. UPDATE THE MODEL CARD



---


#### epoch
- 1 epoch is a complete pass through the entire training dataset
- with n datapoints and batch size = b, n/b iterations to complete 1 epoch
- 1 iteration is a single update of the model's parameters (weights, etc.)


---


#### repo_id, branch, model and datasets repo on HF Hub
- 1 repo = 1 model and 1 tokenizer
- branch = revision



---

#### Averages
<pre>
- per sample: metrics are computed for each sample (= for each instance, = for each row of y_true and y_pred), and then averaged across all samples.
- per label : metrics are computed for each label separately, and then averaged across all labels.
- per batch : metrics are computed for each batch, and then averaged across all batches.

- 'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives.
           Gives more weight to frequent labels → best for imbalanced datasets where frequent labels are more important.
- 'macro': Calculate metrics for each label separately, and find their unweighted mean.
           This does not take label imbalance into account. This is fine for balanced datasets but not for imbalanced datasets since rare labels are given equal weights.
           Averages the metric for each label without considering their imbalance, without considering label frequency.
           When to use: when wanting equal importance for all labels, including rare ones.
           Treats all labels equally → best when you care about rare labels as much as frequent ones.
- 'weighted': Calculate metrics for each label separately, and find their average weighted by support (= the number of true instances for each label). This alters ‘macro’ to account for label imbalance;
              it can result in an F-score that is not between precision and recall.
              Averages per label weighted by their support, without considering label frequency.
              When to use: when wanting to reflect label imbalance (common labels contribute more).
              Like macro but considers label frequency → best if you want a compromise between macro and micro.
- 'samples': Calculate metrics per sample instead of per label, and find their average (only meaningful for multilabel-classification where this differs from accuracy_score).
             Computes the metric per sample and then averages across all samples.
             When to use: when each sample has multiple correct labels.

- 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
- 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
- 'weighted'   AUC: similar to macro but considers label frequency
- 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
- 'weighted PR AUC: also good, but slightly biased toward frequent labels

PR AUC is better than ROC AUC when you care about positive examples in imbalanced data.
</pre>


---

#### learning rate
- A common rule is to scale the learning rate proportionaly with the effective batch size
- **>>>>>>>>>> get_linear_schedule_with_warmup**

#### Path separators
Use os.path.join for constructing paths to ensure correct separators

In [ ]:
THRESHOLD_TUNING = True
ACCESS_TO_HF_HUB = True


#### Hugging Face Hub (HF Hub) authenticate

In [ ]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
api                    = HfApi()
user                   = None

if ACCESS_TO_HF_HUB:
    login(token=hf_token)

    # Check
    user = whoami(token=hf_token)
    assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
class TZFormatter(logging.Formatter):
    def __init__(self, fmt=None, datefmt=None, tzinfo=None):
        super().__init__(fmt=fmt, datefmt=datefmt)
        self.tzinfo = tzinfo
    def converter(self, timestamp):
        return datetime.fromtimestamp(timestamp, self.tzinfo)
    def formatTime(self, record, datefmt=None):
        dt = self.converter(record.created)
        if datefmt:
            return dt.strftime(datefmt)
        else:
            return dt.isoformat()


In [ ]:
zoneInfo  = ZoneInfo("Europe/Brussels")
timestamp = datetime.now(zoneInfo).strftime('%Y%m%d_%H%M%S')

# repo_id (str) = namespace(user or organization)/repo name
repo_id = f"claudelepere/jobs_EN_791112_50000_all_{timestamp}"


In [ ]:
def setup_colab_logging(log_dir='/content/logs', max_mb=10, backups=3):
    """Configure rotating logs with console and file output"""
    os.makedirs(log_dir, exist_ok=True)

    # Create timestamped log file
    log_file = os.path.join(log_dir, f'colab_TrEvTe_{timestamp}.log')

    # Disable Colab's default logging
    logging.getLogger().handlers.clear()  # Remove all default handlers

    logger = logging.getLogger('colab')
    logger.setLevel(logging.DEBUG)  # Capture all levels

    # File handler (rotating)
    fh = RotatingFileHandler(
        log_file,
        maxBytes=max_mb*1024*1024,
        backupCount=backups,
        encoding='utf-8'
    )
    fh.setLevel(logging.DEBUG)

    # Console handler (info level only)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)

    # Formatters
    file_formatter = TZFormatter(
        '%(asctime)s [%(levelname)s] %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        tzinfo=zoneInfo
    )
    console_formatter = TZFormatter(
        '[%(levelname)s] %(message)s',
        datefmt='%H:%M:%S'
    )

    fh.setFormatter(file_formatter)
    ch.setFormatter(console_formatter)

    logger.addHandler(fh)
    logger.addHandler(ch)

    return logger

def download_logs(log_dir='/content/logs'):
    """Download all logs as a zip file"""
    name      = 'TrEvTe_colab_logs'
    zip_path  = f"{log_dir}/{name}.zip"

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, filenames in os.walk(log_dir):
            for f in filenames:
                if f.endswith('.log'):
                    full_path = os.path.join(root, f)
                    arcname = os.path.relpath(full_path, start=log_dir)
                    zipf.write(full_path, arcname)

    api.upload_file(
        path_or_fileobj = zip_path,
        path_in_repo    = f"{name}/{name}.zip",
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    return zip_path


In [ ]:
logger = setup_colab_logging()


In [ ]:
if ACCESS_TO_HF_HUB:
    model_repoUrl   = create_repo(repo_id=repo_id, repo_type="model",   private=True, exist_ok=True)
    dataset_repoUrl = create_repo(repo_id=repo_id, repo_type="dataset", private=True, exist_ok=True)

    #create_branch(repo_id=repo_id, repo_type="model",   branch=branch, exist_ok=True)
    #create_branch(repo_id=repo_id, repo_type="dataset", branch=branch, exist_ok=True)

    logger.info(f"Model Repo Url: {model_repoUrl} created successfully as a private repo")
    logger.info(f"Dataset Repo Url: {dataset_repoUrl} created successfully as a private repo")


[INFO] Model Repo Url: https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_20250727_173135 created successfully as a private repo
[INFO] Dataset Repo Url: https://huggingface.co/datasets/claudelepere/jobs_EN_791112_50000_all_20250727_173135 created successfully as a private repo


In [ ]:
#logger.debug("This is a debug message")        # written to console and file
#logger.info("This is an info message")         # written to file only
#logger.warning("This is a warning message")    # idem
#logger.error("This is an error message")       # idem
#logger.critical("This is a critical message")  # idem

logger.info(f"timestamp: {timestamp}")
logger.info(f"repo_id: {repo_id}")

if ACCESS_TO_HF_HUB:
    logger.info(f"user: {user}")
    #download_logs()

#raise Exception("AAAA")


[INFO] timestamp: 20250727_173135
[INFO] repo_id: claudelepere/jobs_EN_791112_50000_all_20250727_173135
[INFO] user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


### Upload files from local machine to the Colab runtime environment
ml-poc => load_datasetdict_from_zip_to_import.py

In [ ]:
file_to_upload    = "load_datasetdict_from_zip_to_import.py"
uploaded          = files.upload()
uploaded_filename = next(iter(uploaded))

assert uploaded_filename==file_to_upload, f"'{file_to_upload}' required, '{uploaded_filename}' uploaded!"

logger.debug(f"uploaded_filename: {type(uploaded_filename)} {uploaded_filename}")


Saving load_datasetdict_from_zip_to_import.py to load_datasetdict_from_zip_to_import.py


In [ ]:
sys.path.append('/content')
load_datasetdict_from_zip_to_import = importlib.import_module(Path(file_to_upload).stem)

# Check
all_attributes = dir(load_datasetdict_from_zip_to_import)
logger.debug(f"all_attributes: {all_attributes}")


### Convert to native Python types, for JSON and therefore for HF Hub uploads

In [ ]:
def convert_to_native(obj):
    # Dict: convert keys and values
    if isinstance(obj, dict):
        return {
            str(convert_to_native(k)): convert_to_native(v)
            for k, v in obj.items()
        }
    # List or Tuple
    elif isinstance(obj, (list, tuple)):
        return [convert_to_native(i) for i in obj]

    # NumPy scalar
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)

    # NumPy array
    elif isinstance(obj, np.ndarray):
        return convert_to_native(obj.tolist())

    # PyTorch tensor
    elif isinstance(obj, torch.Tensor):
        return convert_to_native(obj.detach().cpu().tolist())

    # Other types (e.g., str, int, float, bool, None)
    else:
        return obj


### Define literal data

#### => HF Hub and <= HF Hub commands
- login, whoami, create_repo, hf_hub_download: HF_hub high-level standalone function
- api.upload_file (low-level HfApi class, there is no standalone)
- tokenizer, model: transformers.tokenizer/model.push_to_hub
- validation_datasetDict: datasets.push_to_hub, dsD=load_dataset

---

1. login, whoami
2. create_repo
3. model_card: api.upload_file
4. labels, id2label: api.upload_file, hf_hub_download
5. validation_datasetDict.push_to_hub, dsD=load_dataset
6. trainer.train: api.upload_file, hf_hub_download
7. tokenizer.push_to_hub, model.push_to_hub
8. Evaluation1: trainer.evaluate: api.upload_file
9. Evaluation2: trainer.predict: api.upload_file
10. Evaluation2: optimized_thresholds: api.upload_file
11. Evaluation3: model.eval: api.upload_file
12. Test1: trainer.evaluate: api.upload_file
13. Test2: trainer.predict: api.upload_file
14. Test3: model.eval: api.upload_file

=> The single flag ACCESS_TO_HF_HUB is enough to control access to an existing HF project.

In [ ]:
skills         = 789101112
all_rows_low   = 0
all_rows_high  = 50000
num_datapoints = all_rows_high - all_rows_low

epochs = 8 #16 #14 #12 #10

logger.info(f"repo_id: {repo_id}")
logger.info(f"timestamp: {timestamp}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"device: {device}")

training_average   = 'micro'
evaluation_average = 'samples'

learning_rate = 2e-5 #1e-5 x 32/8

# default threshold = 0.5
threshold = 0.5

# attention window size
attention_window = 1024 # = max_length = number of tokens per sample

#alfa  = 0.5
#gamma = 4.0
#alpha = 0.75
#gamma = 2.0
alpha = 1.0
gamma = 1.5

threshold_floor = 0.3  # in def predict_with_optimized_thresholds, minimum threshold imposed

multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0


[INFO] repo_id: claudelepere/jobs_EN_791112_50000_all_20250727_173135
[INFO] timestamp: 20250727_173135
[INFO] device: cuda


### OOM = Out Of Memory

#### OOM: use fp16 (half precision) mixed precision training
Reduces memory requirements by up to 50%.

fp16 = True in TrainingArguments

#### OOM: reduce batch size
- small sizes (1 to 32):
  - PROs: better generalization in some cases
  - CONs: may produce noisier gradients
- large sizes (128, 256, or higer):
  - PROs: gradients are smoother, leading to more stable training
  - CONs: poorer generalization (overfitting) in some cases
- intermediate sizes (32, 64): combines the benefits of small and large sizes

In [ ]:
batch_size = 8

#### OOM: enable gradient accumulation

- compensate for smaller batch sizes by accumulating gradients over several steps
- **effective batch size** = per-device batch size x gradient acumulation steps
- in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters

WARNING: gradient_accumulation_steps may not be None => comment it in TrainingArguments

In [ ]:
gradient_accumulation_steps = 4

#### OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

#### OOM: reduce model size or input tokens
- LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
- max_length: 4096 max for Longformer
- a single word can be equal to several tokens; stop words are **NOT discarded**!
- word_text_length_counts_sorted:
      jobs count                 : 50000
      jobs count under  512 words: 44794  89.59%
      jobs count under  640 words: 47894  95.79%
      jobs count under  768 words: 49123  98.25%
      jobs count under  896 words: 49691  99.38%
      jobs count under 1024 words: 49917  99.83%
      jobs count under 2048 words: 50000 100.00%
      jobs count under 4096 words: 50000 100.00%

In [ ]:
#max_length =  768    #      37 min    #
max_length = 1024    #      38 min    # GPU RAM: 12.2 / 40 GB <<<<<<<<<<<< pourrait augmenter à 2048?
#max_length = 2048    # 1 hr 10 min    # GPU RAM: 21.4 / 40 GB
#max_length = 4096    # 2 hr 10 min    # GPU RAM: 39.5 / 40 GB => OutOfMemoryError

#### OOM: limit the number of GPU workers:
* 0 (default) or 1
* in Colab dataloader_num_workers = 1

#### """OOM: reduce the number of transformers layers"""

In [ ]:
"""
hidden_layers = 6  # default:12
"""

'\nhidden_layers = 6  # default:12\n'

#### OOM: check for and kill zombie processes

In [ ]:
!ps aux | grep python
#!kill -9 <PID>
if torch.cuda.is_available():
  !nvidia-smi
  logger.debug(torch.cuda.memory_summary())

root          86  3.9  0.0      0     0 ?        Z    15:26   0:13 [python3] <defunct>
root          87  0.1  0.0  77380 57528 ?        S    15:26   0:00 python3 /usr/local/bin/colab-fileshim.py
root         132  1.2  0.1 797936 135496 ?       Sl   15:27   0:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root         742 15.8  1.5 12407528 1388400 ?    Ssl  15:29   0:32 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-3d5c043f-0e35-423d-928a-577ecbf6e0fe.json
root         776  0.4  0.0 546612 21704 ?        Sl   15:29   0:00 /usr/bin/python3 /usr/local/lib/python3.11/dist-packages/debugpy/adapter --for-server 52783 --host 127.0.0.1 --port 33069 --server-access-token f92dfe95a015cc8d5d5799dea1058128951e805b0fe6cf897b654b8d456d2d96
root        1564  0.2  0.0 1243600 21716 ?       Sl   15:31   0:00 

#### OOM: free up GPU memory

In [ ]:
torch.cuda.empty_cache()

### Hugging Face model card

In [ ]:
# HF model card
# Model card here => README.md on the HF Hub.

model_card = """
---
tags:
- '50000'
---
# Model
Model fine-tuned on higly imbalanced multilabel classification.

## Model details
- Language: English
- Task: Multilabel classification
- Architecture: Longformer
- Pretrained model: [allenai/longformer-base-4096](https://huggingface.co/allenai/longformer-base-4096)
- Framework: Pytorch
- Version 1.0.0

## Training Data
- skills: 7, 9, 11, 12
- 50000 job datapoints

## Fine-tuning parameters
- batch size: 8
- gradient accumulation: 4
- fp16 precision
- input tokens max length: 1024
- epochs: 8
- learning rate: 2e-5
- attention window size: 1024
- training average: micro
- evaluation average: samples
- threshold tuning: True
- threshold: 0.5
- focal loss: alpha=1.0, gamma=1.5
- threshold_floor: 0.3: NO
- exact match optimized thresholds: YES
"""

### Input
**NO_PUSH** =>
1. dataset_EN_791112_0_50000.zip


In [ ]:
datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}.zip"
#if above_threshold:
#    datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_above.zip"
#else:
#    datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_below.zip"

#datasetDict_zip_file_name = "dataset_EN_791112_0_50000_all.zip"
#datasetDict_zip_file_name = "dataset_EN_9_0_120.zip"

datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]  # NOT USED

logger.debug(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
logger.debug(f"datasetDict_dir_name     : {datasetDict_dir_name}")


### Output

=> **HF Hub**
- Model Repo Url: https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_mmddhh
- Dataset Repo Url: https://huggingface.co/datasets/claudelepere/jobs_EN_791112_50000_all_mmddhh
- Model card => https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_mmddhh
___
=> **HF Hub model repo**
1. labels = 'labels', 'id2label'
2. tokenizer
3. model
4. optimized_thresholds
5. trainer_train
6. evaluation_trainer_evaluate
7. evaluation_trainer_predict
8. evaluation_model_eval
9. test_trainer_evaluate
10. test_trainer_predict
11. test_model_eval

=> **HF Hub dataset repo**
1. validation_dataset

In [ ]:
"""
encoded_validation_dataset_tmp        = "encoded_validation_dataset_tmp"
encoded_validation_dataset_zip        = "encoded_validation_dataset.zip"
encoded_validation_dataset            = os.path.splitext(encoded_validation_dataset_zip)[0]
#encoded_validation_dataset_drive_path = "/content/drive/MyDrive/encoded_validation_dataset"
#encoded_validation_dataset_file_path  = f"{encoded_validation_dataset_drive_path}/{encoded_validation_dataset_zip}"


## Ensure the directory exists before saving the dataset and making the archive
#os.makedirs(encoded_validation_dataset_tmp, exist_ok=True)


logger.debug(f"encoded_validation_dataset_tmp : {encoded_validation_dataset_tmp}")
logger.debug(f"encoded_validation_dataset_zip : {encoded_validation_dataset_zip}")
logger.debug(f"encoded_validation_dataset     : {encoded_validation_dataset}")
#logger.debug(f"encoded_validation_dataset_drive_path      : {encoded_validation_dataset_drive_path}")
#logger.debug(f"encoded_validation_dataset_file_path       : {encoded_validation_dataset_file_path}")
"""

'\nencoded_validation_dataset_tmp        = "encoded_validation_dataset_tmp"\nencoded_validation_dataset_zip        = "encoded_validation_dataset.zip"\nencoded_validation_dataset            = os.path.splitext(encoded_validation_dataset_zip)[0]\n#encoded_validation_dataset_drive_path = "/content/drive/MyDrive/encoded_validation_dataset"\n#encoded_validation_dataset_file_path  = f"{encoded_validation_dataset_drive_path}/{encoded_validation_dataset_zip}"\n\n\n## Ensure the directory exists before saving the dataset and making the archive\n#os.makedirs(encoded_validation_dataset_tmp, exist_ok=True)\n\n\nlogger.debug(f"encoded_validation_dataset_tmp : {encoded_validation_dataset_tmp}")\nlogger.debug(f"encoded_validation_dataset_zip : {encoded_validation_dataset_zip}")\nlogger.debug(f"encoded_validation_dataset     : {encoded_validation_dataset}")\n#logger.debug(f"encoded_validation_dataset_drive_path      : {encoded_validation_dataset_drive_path}")\n#logger.debug(f"encoded_validation_dataset

### """Delete a file if present"""

In [ ]:
"""
def delete_local_file(file_path):
    try:
        file_path.unlink(missing_ok=True)  # missing_ok=True prevents exception if file doesn't exist
        logger.debug(f"Successfully deleted file: {file_path}")
    except PermissionError:
        logger.debug(f"Permission denied to delete file: {file_path} (File might be in use by another program)")
    except Exception as e:
        logger.debug(f"Error deleting file {file_path}: {e}")
"""

'\ndef delete_local_file(file_path):\n    try:\n        file_path.unlink(missing_ok=True)  # missing_ok=True prevents exception if file doesn\'t exist\n        logger.debug(f"Successfully deleted file: {file_path}")\n    except PermissionError:\n        logger.debug(f"Permission denied to delete file: {file_path} (File might be in use by another program)")\n    except Exception as e:\n        logger.debug(f"Error deleting file {file_path}: {e}")\n'

### """Delete encoded_validation_dataset.zip in NO_PUSH if present"""

IT'S IMPOSSIBLE IN COLAB!
When you run code in Colab, it executes on Google's servers (in the cloud), not on your local PC - unless you specifically set up a connection between them.

files.download() in Colab sends files to your local PC, but the opposite operation - deleting local files from Colab - isn't directly possible because of security restrictions in web browsers.

Why you can download but not delete locally?
- Downloads are initiated by your browser (you choose where to save)
- File deletion would require access to your local file system, which browsers strictly prohibit for security reasons. Colab isn't meant to manage local files.

**files.download()** is less stable and reliable than Google Drive

In [ ]:
"""
delete_local_file(encoded_validation_dataset_zip_NO_PUSH_path)
"""

'\ndelete_local_file(encoded_validation_dataset_zip_NO_PUSH_path)\n'

### """Mount Google Drive"""
drive.mount('/content/drive') gives access to your Google Drive under the path /content/drive/MyDrive/

### """Google Cloud Authentication"""

In [ ]:
"""
auth.authenticate_user()  # user = c.lepere@ictjob.be
"""

'\nauth.authenticate_user()  # user = c.lepere@ictjob.be\n'

In [ ]:
"""
mountpoint = '/content/drive'

# Check whether the directory is not empty and remove its contents
if os.path.isdir(mountpoint) and os.listdir(mountpoint):
    logger.debug(f"Mountpoint '{mountpoint}' is not empty. Removing contents...")
    for item in os.listdir(mountpoint):
        item_path = os.path.join(mountpoint, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            logger.debug(f"Removed file: {item_path}")
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
            logger.debug(f"Removed directory: {item_path}")
    logger.debug(f"Contents of '{mountpoint}'removed successfully.")
else:
    logger.debug(f"Mountpoint '{mountpoint}' is empty or does not exist.")

# Now proceed with mounting the drive
drive.mount(mountpoint)
# os.makedirs is required at both the source and destination but after drive.mount
os.makedirs(encoded_validation_dataset_drive_path, exist_ok=True)
"""

'\nmountpoint = \'/content/drive\'\n\n# Check whether the directory is not empty and remove its contents\nif os.path.isdir(mountpoint) and os.listdir(mountpoint):\n    logger.debug(f"Mountpoint \'{mountpoint}\' is not empty. Removing contents...")\n    for item in os.listdir(mountpoint):\n        item_path = os.path.join(mountpoint, item)\n        if os.path.isfile(item_path):\n            os.remove(item_path)\n            logger.debug(f"Removed file: {item_path}")\n        elif os.path.isdir(item_path):\n            shutil.rmtree(item_path)\n            logger.debug(f"Removed directory: {item_path}")\n    logger.debug(f"Contents of \'{mountpoint}\'removed successfully.")\nelse:\n    logger.debug(f"Mountpoint \'{mountpoint}\' is empty or does not exist.")\n\n# Now proceed with mounting the drive\ndrive.mount(mountpoint)\n# os.makedirs is required at both the source and destination but after drive.mount\nos.makedirs(encoded_validation_dataset_drive_path, exist_ok=True)\n'

### '''Logging'''

In [ ]:
'''
# Is /content the current directory?
logger.debug(f"Current directory: {os.getcwd()}")

# Ensure the logs directory exists
os.makedirs('/content/logs', exist_ok=True)

# Create handlers for both file and console output
file_handler = logging.FileHandler('/content/logs/training.log')
file_handler.setLevel(logging.INFO)                               # Log level for file

#console_handler = logging.StreamHandler()
#console_handler.setLevel(logging.INFO)                            # Log level for console

# Create a formatter and attach it to both handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
#console_handler.setFormatter(formatter)

# Add the handlers to the root logger
logger = logging.getLogger()
#logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
#logger.addHandler(console_handler)

#transformers_logging.set_verbosity_error()  # Only show errors, suppress info

# Remove all existing handlers of type StreamHandler
#for handler in logger.handlers[:]:
#    if isinstance(handler, logging.StreamHandler):
#        logger.removeHandler(handler)

# Verify by printing current handlers
#logger.debug("handlers", logger.handlers)  # Should only show FileHandler

# Test logging to ensure both file and console work
logging.info("Testing log before training starts.")
'''

'\n# Is /content the current directory?\nlogger.debug(f"Current directory: {os.getcwd()}")\n\n# Ensure the logs directory exists\nos.makedirs(\'/content/logs\', exist_ok=True)\n\n# Create handlers for both file and console output\nfile_handler = logging.FileHandler(\'/content/logs/training.log\')\nfile_handler.setLevel(logging.INFO)                               # Log level for file\n\n#console_handler = logging.StreamHandler()\n#console_handler.setLevel(logging.INFO)                            # Log level for console\n\n# Create a formatter and attach it to both handlers\nformatter = logging.Formatter(\'%(asctime)s - %(levelname)s - %(message)s\')\nfile_handler.setFormatter(formatter)\n#console_handler.setFormatter(formatter)\n\n# Add the handlers to the root logger\nlogger = logging.getLogger()\n#logger.setLevel(logging.INFO)\nlogger.addHandler(file_handler)\n#logger.addHandler(console_handler)\n\n#transformers_logging.set_verbosity_error()  # Only show errors, suppress info\n\n# Rem

### Save locally, then upload model card to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name            = "model_card"
    model_card_path = f"{name}.md"

    with open(model_card_path, "w") as f:
        f.write(model_card)

    api.upload_file(
        path_or_fileobj = model_card_path,
        path_in_repo    = 'README.md',
        repo_id         = repo_id,
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {model_card}")


[INFO] model_card successfully uploaded to HF Hub as model_card.md


In [ ]:
"""
# Check the Python version
logger.debug(sys.version)
logger.debug("__________________________________________________________________")

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
logger.debug("__________________________________________________________________")


# Check system information
!cat /etc/os-release
!uname -m
logger.debug("__________________________________________________________________")


# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#logger.debug("__________________________________________________________________")


# Check RAM
!free -h
logger.debug("__________________________________________________________________")


# Check disk space
!df -h
logger.debug("__________________________________________________________________")


# Get environment variables
for key, value in os.environ.items():
    logger.debug(f"{key}: {value}")
"""
!python -V
logger.debug(f"currentdir: {os.getcwd()}")

Python 3.11.13


### NO_PUSH => dataset_EN_791112_0_50000.zip

In [ ]:
import os
import shutil
from google.colab import files

def upload_and_unpack(target_dir="/content/unpacked"):
    uploaded = files.upload()
    archive_path = next(iter(uploaded))

    os.makedirs(target_dir, exist_ok=True)

    shutil.unpack_archive(archive_path, target_dir)

    print(f"Archive unpacked to: {target_dir}")

    return target_dir

In [ ]:
folder = upload_and_unpack("/content/dataset_EN_789101112_0_50000")

Saving dataset_EN_789101112_0_50000.zip to dataset_EN_789101112_0_50000.zip
Archive unpacked to: /content/dataset_EN_789101112_0_50000


In [ ]:
from google.colab import files

# Upload via browser
uploaded = files.upload()
zip_filename = next(iter(uploaded))

# ✅ Change this if your file has a different name
#ip_filename = datasetDict_zip_file_name

target_dir = "/content/dataset_EN_789101112_0_50000"
os.makedirs(target_dir, exist_ok=True)

Saving dataset_EN_9_0_120.zip to dataset_EN_9_0_120.zip


In [ ]:
# 📌 STEP 3: Move and unzip into /content/uploaded_files
#!mkdir /content/dataset_EN_789101112_0_50000/
os.makedirs("/content/dataset_EN_789101112_0_50000", exist_ok=True)
#!mv $zip_filename /content/uploaded_files/
!unzip -o /uploaded_files/$zip_filename -d /content/dataset_EN_789101112_0_50000/

Archive:  /content/uploaded_files/dataset_EN_9_0_120.zip
  inflating: /content/uploaded_files/dataset_EN_9_0_120/dataset_dict.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/test/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/test/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/test/state.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/train/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/train/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/train/state.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/validation/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/validation/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_9_0_120/validation/state.json  


In [ ]:
dataset_dict = load_datasetdict_from_zip_to_import.setup_and_load_dataset(
    subdir_name  = "uploaded_files",
    zip_filename = zip_filename
)

# ✅ DatasetDict ready
logger.debug(f"dataset_dict              : {type(dataset_dict)} shape={dataset_dict.shape}\n{dataset_dict}")
# <class 'datasets.dataset_dict.DatasetDict'> shape={'train': (58, 83), 'validation': (7, 83), 'test': (8, 83)}
logger.debug(f"dataset_dict.keys()       : {dataset_dict.keys()}")
# dict_keys(['train', 'validation', 'test'])
logger.debug(f"dataset_dict['train']     : {type(dataset_dict['train'])} shape={dataset_dict['train'].shape}")
# <class 'datasets.arrow_dataset.Dataset'> shape=(58, 83)
logger.debug(f"dataset_dict['validation']: {type(dataset_dict['validation'])} shape={dataset_dict['validation'].shape}")
# <class 'datasets.arrow_dataset.Dataset'> shape=(7, 83)
logger.debug(f"dataset_dict['test']      : {type(dataset_dict['test'])} shape={dataset_dict['test'].shape}")
# <class 'datasets.arrow_dataset.Dataset'> shape=(8, 83)

#raise Exception("BBBB")


🔍 Looking for dataset at: /content/uploaded_files/dataset_EN_9_0_120
📂 Loading DatasetDict from: /content/uploaded_files/dataset_EN_9_0_120
✅ Loaded DatasetDict dataset_dict.


### """W&B initialization (not used now)"""

In [ ]:
"""
run_name = f"EN_{skills}_{all_rows_low}_{all_rows_high}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{fp}"

if 'gradient_accumulation_steps' not in globals():
  run_name = f"{run_name}_ba{batch_size}"
else:
  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"

print(f"run_name: {run_name}")
"""

'\nrun_name = f"EN_{skills}_{all_rows_low}_{all_rows_high}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{fp}"\n\nif \'gradient_accumulation_steps\' not in globals():\n  run_name = f"{run_name}_ba{batch_size}"\nelse:\n  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"\n\nprint(f"run_name: {run_name}")\n'

In [ ]:
"""
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)
"""

'\nos.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ\nwandb_api_key               = os.environ.get(\'WANDB_API_KEY\')\nwandb.login(key=wandb_api_key)\n'

In [ ]:
"""
try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : epochs,
          "batch_size"   : batch_size
      }
  )
except wandb.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")
"""

'\ntry:\n  wandb.init(\n      project = "skill_classification",\n      name    = run_name,\n      entity  = "claudelepere-c-cile-cy",\n      config  = {\n          "learning_rate": learning_rate,\n          "epochs"       : epochs,\n          "batch_size"   : batch_size\n      }\n  )\nexcept wandb.CommError as err:\n  print(f"CommError: {err}")\nexcept Exception as exc:\n  print(f"Exception: {exc}")\n'

In [ ]:
example = dataset_dict['train'][0]

print(f"dataset_dict['train'][0]: {type(example)} {example.keys()}\n{example}")


dataset_dict['train'][0]: <class 'dict'> dict_keys(['id', 'text', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '827', '828', '829', '830', '358', '359', '365', '367', '369', '379', '382', '385', '409', '810'])
{'id': 335819, 'text': "Innova - API GateWay Engineer - B2B   Axway API GW, Apigee F5-ASM, REST   Innova   Our client is a prominent player in the financial sector, recognized for its comprehensive range of banking and financial services. With a global presence, it serves a diverse clientele, including individuals, corporations, and institutions. Emphasizing innov

### Create labels (list of str), id2label (int => str) and label2id (str => int).

<mark>Note: there are labels and labels: DO NOT CONFUSE!</mark>
1. labels extracted from the original dataset
   DatasetDict({
    train: Dataset({
        features: ['id', 'text', '610', '611', etc., '409', '810'],
        num_rows: 58
    })
2. further, labels column of the encoded_dataset
   DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 58
    })

* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [ ]:
labels     = [label for label in dataset_dict['train'].features.keys() if label not in ['id', 'text']]
num_labels = len(labels)
id2label   = {idx: label for idx, label in enumerate(labels)}
label2id   = {label: idx for idx, label in enumerate(labels)}

logger.debug(f"labels    : {type(labels)}   len={len(labels)}\n{labels}")
logger.debug(f"num_labels: {num_labels}")
logger.debug(f"id2label  : {type(id2label)} len={len(id2label)}\n{id2label}")
logger.debug(f"label2id  : {type(label2id)} len={len(label2id)}\n{label2id}")

labels_key_types_set     = {type(key) for key in labels}
id2label_key_types_set   = {type(key) for key in id2label}
id2label_value_types_set = {type(value) for value in id2label.values()}
label2id_key_types_set   = {type(key) for key in label2id}
logger.debug(f"labels_key_types_set    : {type(labels_key_types_set)}     len={len(labels_key_types_set)}     {labels_key_types_set}")      # labels_key_types_set  : len=1   {<class 'str'>}
logger.debug(f"id2label_key_types_set  : {type(id2label_key_types_set)}   len={len(id2label_key_types_set)}   {id2label_key_types_set}")    # id2label_key_types_set: len=1 {<class 'int'>}
logger.debug(f"id2label_value_types_set: {type(id2label_value_types_set)} len={len(id2label_value_types_set)} {id2label_value_types_set}")  # id2label_value_types_set: len=1 {<class 'str'>}
logger.debug(f"label2id_key_types_set  : {type(id2label_key_types_set)}   len={len(label2id_key_types_set)}   {label2id_key_types_set}")    # label2id_key_types_set: len=1 {<class 'str'>}

#raise Exception("CCCC")


### labels and id2label => HF Hub

WARNING:
- labels_dataset = Dataset.from_dict({'labels': labels, 'id2label': id2label}) creates a Dataset, not a DatasetDict.
- Storing id2label, a Python dictionary, as columns is not typical and can cause serialization problems or odd behavior.
- <mark>Due to the JSON conversion required for uploading to HF Hub, id2label (item of the dict labels) in HF Hub is a dict of str: str and it is also a dict of str: str when downloaded (item of the dict labels),<br>
unlike labels that was and remains a list of str</mark>

In [ ]:
labels_native   = convert_to_native(labels)
id2label_native = convert_to_native(id2label)

if ACCESS_TO_HF_HUB:
    name         = "labels_native"
    labels_path  = f"{name}.json"
    path_in_repo = f"{name}/{labels_path}"

    with open(labels_path, "w") as f:
        json.dump(
        {
            'labels_native': labels_native,
            'id2label_native': id2label_native
        }, f)

    api.upload_file(
        path_or_fileobj = labels_path,
        path_in_repo    = path_in_repo,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {path_in_repo}")

    downloaded_file_path = hf_hub_download(
        repo_id   = repo_id,
        repo_type = 'model',
        filename  = path_in_repo
    )
    logger.debug(f"labels_path: {type(downloaded_file_path)} len={len(downloaded_file_path)}\n{downloaded_file_path}")

    with open(downloaded_file_path, "r") as f:
        labels_dict_native_W = json.load(f)  # W as work
    logger.debug(f"labels_dict_native_W['labels_native']  : {type(labels_dict_native_W['labels_native'])}   len={len(labels_dict_native_W['labels_native'])}  \n{labels_dict_native_W['labels_native']}")
    logger.debug(f"labels_dict_native_W['id2label_native']: {type(labels_dict_native_W['id2label_native'])} len={len(labels_dict_native_W['id2label_native'])}\n{labels_dict_native_W['id2label_native']}")

    #raise Exception("DDDD")

[INFO] labels_native successfully uploaded to HF Hub as labels_native/labels_native.json


labels_native.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

### Load pretrained tokenizer and model

In [ ]:
model_name = "allenai/longformer-base-4096"


In [ ]:
# Increase the download timeout for Hugging Face Hub operations
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '60'
tokenizer                             = LongformerTokenizerFast.from_pretrained(model_name)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
# Check the type of the keys
logger.debug(f"id2label_key_types_set: len={len(id2label_key_types_set)} {id2label_key_types_set}")  # id2label_key_types_set: len=1 {<class 'int'>}
logger.debug(f"label2id_key_types_set: len={len(label2id_key_types_set)} {label2id_key_types_set}")  # label2id_key_types_set: len=1 {<class 'str'>}

model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels   = num_labels,
    id2label     = id2label,
    label2id     = label2id,
    problem_type = 'multi_label_classification'
)

# Configure attention window size
model.config.attention_window = attention_window


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Optimizer Initialization
- optimizer: will be used in the training loop to update the model's parameters.
- torch.optim.AdamW: This specifies the type of optimizer being used. AdamW is a popular optimization algorithm in deep learning. It's a variation of the Adam optimizer that includes "weight decay" (the "W"). Weight decay is a regularization technique that helps prevent the model from overfitting to the training data by penalizing large weights.
- model.parameters(): This is the first argument passed to the AdamW optimizer. It's a method provided by PyTorch Module (which your model object inherits from) that returns an iterator over all the trainable parameters (like weights and biases) of the model. The optimizer needs to know which parameters it should update during training.
- lr=learning_rate: This is the second argument, lr stands for "learning rate". It's a hyperparameter that controls the step size at each iteration while moving towards a minimum of the loss function. A smaller learning rate means smaller steps, which can lead to more stable training but might take longer. A larger learning rate means larger steps, which can speed up training but might overshoot the minimum or lead to unstable training.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


### Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

In [ ]:
def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #logger.debug(f"created labels_matrix: {type(labels_matrix)} shape={labels_matrix.shape}\n{labels_matrix}")

  #---------Populate label matrix
  for idx, label in enumerate(labels):
    # idx:0 label:142 idx:1 label:143 ... idx:145 label:352 idx:146 label:353
    #logger.debug(f"idx:{idx} label:{label}")
    if label in examples:
        labels_matrix[:, idx] = torch.tensor(
            [1.0 if val else 0.0 for val in examples[label]],
            dtype=torch.float32
        )
  # labels_matrix: <class 'torch.Tensor'> shape=torch.Size([1000, 147])
  # tensor([[0., 0., 0.,  ..., 0., 1., 0.],
  #      ...,
  #      [0., 0., 0.,  ..., 1., 0., 0.]])
  #logger.debug(f"labels_matrix: {type(labels_matrix)} shape={labels_matrix.shape}\n{labels_matrix}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  # encoding['labels']: <class 'torch.Tensor'> shape=torch.Size([1000, 147])
  #logger.debug(f"encoding['labels']: {type(encoding['labels'])} shape={encoding['labels'].shape}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  #logger.debug(f"1 preprocess_data call: encoding: {type(encoding)} {encoding.keys()}")

  return encoding


### Create encoded_dataset (datasets.dataset_dict.DatasetDict) = 3 encoded datasets.arrow_dataset.Dataset, train, validation and test

In [ ]:
encoded_dataset = dataset_dict.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset_dict['train'].column_names,
    with_indices   = True
)
logger.debug(f"encoded_dataset: {type(encoded_dataset)} shape={encoded_dataset.shape}\n{encoded_dataset}")
logger.debug("_________________________________________________________________________")

logger.debug(f"encoded_dataset.keys(): {type(encoded_dataset.keys())} len={len(encoded_dataset.keys())} {encoded_dataset.keys()}")
# encoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> shape={'train': (58, 4), 'validation': (7, 4), 'test': (8, 4)}
logger.debug(f"encoded_dataset['train']: {type(encoded_dataset['train'])} shape={encoded_dataset['train'].shape}")
# <class 'datasets.arrow_dataset.Dataset'> shape=(58, 4)
logger.debug(f"encoded_dataset['train'].column_names: {type(encoded_dataset['train'].column_names)} len={len(encoded_dataset['train'].column_names)} {encoded_dataset['train'].column_names}")
# <class 'list'> len=4 ['input_ids', 'attention_mask', 'global_attention_mask', 'labels']
logger.debug(f"encoded_dataset['train'].features: {encoded_dataset['train'].features}")
#{'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'global_attention_mask': List(Value('int64')), 'labels': List(Value('float32'))}
logger.debug(f"encoded_dataset['train'][0]: {type(encoded_dataset['train'][0])}\n{encoded_dataset['train'][0]}")
# encoded_dataset['train'][0]: <class 'dict'>
# {'input_ids': [0, 1121, etc.], 'attention_mask': [1, 1, etc., 0, 0], 'global_attention_mask': [1, 0, etc., 0, 0], 'labels': [0.0, 0.0, etc., 1.0, 1.0, 0.0, etc., 0.0, 0.0]}

#raise Exception("EEEE")


Map:   0%|          | 0/58 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

### Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines
Format the 3 HF datasets as PyTorch datasets?

No! They are not converted to PyTorch tensors:

- train_dataset: <class 'datasets.arrow_dataset.Dataset'>
- train_dataset['input_ids']: <class 'datasets.arrow_dataset.Column'>
- train_dataset['attention_mask']: <class 'datasets.arrow_dataset.Column'>
- train_dataset['global_attention_mask']: <class 'datasets.arrow_dataset.Column'>
- train_dataset['labels']: <class 'datasets.arrow_dataset.Column'>

with_format('torch') only sets a *formatting preference*, it does not convert.  
The library remains lazy—it won't convert to PyTorch tensors until you actually access the data in a way that requires it.

<mark>From now on, I won't use the name “encoded_dataset” but the names “train/validation/test_dataset”.</mark>

In [ ]:
# Without .with_format('torch')
train_dataset      = encoded_dataset['train']
logger.debug(f"train_dataset['input_ids'][0]: {type(train_dataset['input_ids'][0])}\tlen={len(train_dataset['input_ids'][0])}\n{train_dataset['input_ids'][0]}")
# <class 'list'>	len=1024

logger.debug("_________________________________________________________________________")

# With .with_format('torch')
train_dataset      = encoded_dataset['train'].with_format('torch')
validation_dataset = encoded_dataset['validation'].with_format('torch')
test_dataset       = encoded_dataset['test'].with_format('torch')

logger.debug("__________________________________________________________________")
logger.debug(f"train_dataset:                 {type(train_dataset)} {train_dataset.shape}\t{train_dataset.features}\n{train_dataset}")
logger.debug(f"validation_dataset:            {type(validation_dataset)} {validation_dataset.shape}\t{validation_dataset.features}\n{validation_dataset}")
# <class 'datasets.arrow_dataset.Dataset'> (58, 4)	{'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'global_attention_mask': List(Value('int64')), 'labels': List(Value('float32'))}
# <class 'datasets.arrow_dataset.Dataset'> (7, 4)	{'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'global_attention_mask': List(Value('int64')), 'labels': List(Value('float32'))}
logger.debug(f"train_dataset['input_ids'][0]: {type(train_dataset['input_ids'][0])}\tlen={len(train_dataset['input_ids'][0])}\tshape={train_dataset['input_ids'][0].shape}\n{train_dataset['input_ids'][0]}")
# <class 'torch.Tensor'>	len=1024	shape=torch.Size([1024]
logger.debug(f"train_dataset['input_ids'][:]: {type(train_dataset['input_ids'][:])}\tlen={len(train_dataset['input_ids'][:])}\tshape={train_dataset['input_ids'][:].shape}\n{train_dataset['input_ids'][:]}")
# <class 'torch.Tensor'>	len=58	shape=torch.Size([58, 1024])

logger.debug("_________________________________________________________________________")
logger.debug(f"train_dataset['input_ids']:             {type(train_dataset['input_ids'])}\tlen={len(train_dataset['input_ids'])}\n{train_dataset['input_ids']}")
# <class 'datasets.arrow_dataset.Column'>	len=58
logger.debug(f"train_dataset['attention_mask']:        {type(train_dataset['attention_mask'])}\tlen={len(train_dataset['attention_mask'])}\n{train_dataset['attention_mask']}")
# <class 'datasets.arrow_dataset.Column'>	len=58
logger.debug(f"train_dataset['global_attention_mask']: {type(train_dataset['global_attention_mask'])}\tlen={len(train_dataset['global_attention_mask'])}\n{train_dataset['global_attention_mask']}")
# <class 'datasets.arrow_dataset.Column'>	len=58
logger.debug(f"train_dataset['labels']:                {type(train_dataset['labels'])}\tlen={len(train_dataset['labels'])}\n{train_dataset['labels']}")
# <class 'datasets.arrow_dataset.Column'>	len=58

#raise Exception("FFFF")


### """Save Encoded Validation Dataset to Google Drive"""

In [ ]:
"""
# Save to disk and zip
validation_dataset.save_to_disk(encoded_validation_dataset_tmp)
shutil.make_archive(encoded_validation_dataset, 'zip', encoded_validation_dataset_tmp)

# Save to Google Drive
!cp {encoded_validation_dataset_zip} {encoded_validation_dataset_file_path}

# Download to NO_PUSH
#files.download(encoded_validation_dataset_zip)

#logger.debug("encoded_validation_dataset_zip successfully downloaded")
#raise Exception("STOP HERE")
"""

'\n# Save to disk and zip\nvalidation_dataset.save_to_disk(encoded_validation_dataset_tmp)\nshutil.make_archive(encoded_validation_dataset, \'zip\', encoded_validation_dataset_tmp)\n\n# Save to Google Drive\n!cp {encoded_validation_dataset_zip} {encoded_validation_dataset_file_path}\n\n# Download to NO_PUSH\n#files.download(encoded_validation_dataset_zip)\n\n#logger.debug("encoded_validation_dataset_zip successfully downloaded")\n#raise Exception("STOP HERE")\n'

#### """Check retrieval of encoded_validation_dataset.zip"""

In [ ]:
"""
#uploaded = files.upload()

#with zipfile.ZipFile(encoded_validation_dataset_zip, 'r') as zip_ref:
#    zip_ref.extractall(encoded_validation_dataset)

#encoded_validation_datasetZZZ = load_from_disk(encoded_validation_dataset)
#logger.debug(f"encoded_validation_datasetZZZ: {type(encoded_validation_datasetZZZ)} {encoded_validation_datasetZZZ}")

check_file_path = f"{encoded_validation_dataset_drive_path}/check.zip"
logger.debug(f"check_file_path: {check_file_path}")
!cp {encoded_validation_dataset_file_path} {check_file_path}

with zipfile.ZipFile(check_file_path, "r") as zip_ref:
    zip_ref.extractall('encoded_validation_dataset')

encoded_validation_dataset_ZZZ = load_from_disk('encoded_validation_dataset')
logger.debug(f"encoded_validation_dataset_ZZZ: {type(encoded_validation_dataset_ZZZ)} {encoded_validation_dataset_ZZZ.shape}\n{encoded_validation_dataset_ZZZ}")
#raise Exception("STOP")
"""

'\n#uploaded = files.upload()\n\n#with zipfile.ZipFile(encoded_validation_dataset_zip, \'r\') as zip_ref:\n#    zip_ref.extractall(encoded_validation_dataset)\n\n#encoded_validation_datasetZZZ = load_from_disk(encoded_validation_dataset)\n#logger.debug(f"encoded_validation_datasetZZZ: {type(encoded_validation_datasetZZZ)} {encoded_validation_datasetZZZ}")\n\ncheck_file_path = f"{encoded_validation_dataset_drive_path}/check.zip"\nlogger.debug(f"check_file_path: {check_file_path}")\n!cp {encoded_validation_dataset_file_path} {check_file_path}\n\nwith zipfile.ZipFile(check_file_path, "r") as zip_ref:\n    zip_ref.extractall(\'encoded_validation_dataset\')\n\nencoded_validation_dataset_ZZZ = load_from_disk(\'encoded_validation_dataset\')\nlogger.debug(f"encoded_validation_dataset_ZZZ: {type(encoded_validation_dataset_ZZZ)} {encoded_validation_dataset_ZZZ.shape}\n{encoded_validation_dataset_ZZZ}")\n#raise Exception("STOP")\n'

### validation_dataset => HF Hub

In [ ]:
validation_datasetDict = DatasetDict({'validation': validation_dataset})
logger.debug(f"validation_datasetDict: {type(validation_datasetDict)} shape={validation_datasetDict.shape}\n{validation_datasetDict}")
# <class 'datasets.dataset_dict.DatasetDict'> shape={'validation': (7, 4)}

validation_dataset     = validation_datasetDict['validation']
logger.debug(f"validation_dataset: {type(validation_dataset)} shape={validation_dataset.shape}\n{validation_dataset}")
# <class 'datasets.arrow_dataset.Dataset'> shape=(7, 4)

logger.debug(f"validation_dataset['input_ids'][:]: {type(validation_dataset['input_ids'][:])} len={len(validation_dataset['input_ids'][:])} shape={validation_dataset['input_ids'][:].shape}\n{validation_dataset['input_ids'][:]}")
# <class 'torch.Tensor'> len=7 shape=torch.Size([7, 1024])

validation_datasetDict.push_to_hub(
    repo_id,
    config_name = 'validation_dataset' # This will create the directory on the hub
)

# Check
# Load the dataset from the specified repo_id and config_name
# The config_name is handled internally by load_dataset when loading from the hub
# You don't need to specify config_name again here if you are loading
# the configuration that was created with that config_name during push.
dsD = load_dataset(repo_id, 'validation_dataset')
logger.debug(f"dsD: {type(dsD)} len={len(dsD)} shape={dsD.shape}\n{dsD}")
# <class 'datasets.dataset_dict.DatasetDict'> len=1 shape={'validation': (7, 4)}

ds = dsD['validation']
logger.debug(f"ds: {type(ds)} len={len(ds)} shape={ds.shape}\n{ds}")
# <class 'datasets.arrow_dataset.Dataset'> len=7 shape=(7, 4)

logger.info(f"validation_dataset successfully uploaded to HF Hub datasets/{repo_id}")

#raise Exception("GGGG")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########| 12.5kB / 12.5kB            

README.md:   0%|          | 0.00/465 [00:00<?, ?B/s]

validation_dataset/validation-00000-of-0(…):   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7 [00:00<?, ? examples/s]

[INFO] validation_dataset successfully uploaded to HF Hub as claudelepere/jobs_EN_791112_50000_all_20250727_173135/validation_dataset


### example_text = dataset_dict['train'][0]['text']

#### 1) truncated part on dataset_dict['train'][0]['text']

In [ ]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
  )
  logger.debug(f"tokens: {type(tokens)} len={len(tokens)}\n{tokens}")
  # <class 'transformers.tokenization_utils_base.BatchEncoding'> len=3
  logger.debug(f"tokens.keys(): {tokens.keys()}")
  # KeysView({'input_ids': [[0, 1121, 38823, 111, 21013, 14552, 24450, 24379, 111, 163, 176, 387, 1437, 1437, 14249, 1970, 21013, 26204, 6, 8712, 1023, 1942, 274, 245, 12, 2336, 448, 6, 40746, 1437, 1437, 9548, 3900, 1437, 1437, 1541, 3653, 16, 10, 5395, 869, 11, 5, 613, 1293, 6, 4984, 13, 63, 5145, 1186, 9, 3454, 8, 613, 518, 4, 590, 10, 720, 2621, 6, 24, 4542, 10, 5544, 3653, 6902, 6, 217, 2172, 6, 9053, 6, 8, 3353, 4, 3676, 32934, 2787, 4695, 8, 2111, 12, 19483, 1571, 6, 24, 15178, 3443, 3319, 4233, 7, 2438, 17719, 2643, 14, 972, 5, 14007, 782, 9, 63, 7193, 4, 287, 41, 21013, 14552, 24450, 24379, 36, 387, 176, 387, 238, 624, 3303, 1848, 9850, 518, 25323, 40, 173, 561, 19, 5, 2210, 3748, 22820, 13, 5, 3165, 1521, 6, 8809, 6, 278, 12, 658, 6, 2683, 1938, 6, 1052, 6, 323, 8, 14877, 9, 5, 3653, 18, 21013, 13469, 4, 370, 40, 2179, 8, 3014, 26204, 573, 1986, 13, 1533, 2111, 7872, 624, 41, 5693, 3303, 1848, 12723, 6447, 13, 5, 21013, 1753, 1761, 4, 370, 40, 173, 2116, 12, 35435, 2368, 19, 25521, 6, 19168, 6, 8, 2090, 4749, 5019, 6, 420, 1533, 893, 4, 653, 47, 581, 109, 7438, 6, 2438, 8, 323, 5, 7465, 10490, 227, 1633, 1743, 8, 265, 2975, 624, 7664, 11534, 13, 163, 176, 387, 6, 163, 176, 347, 8, 6494, 335, 2081, 2643, 4, 274, 1975, 9764, 15, 5, 7465, 9, 3748, 2502, 11, 5, 6494, 13469, 13, 5, 6979, 3917, 594, 25, 157, 25, 13, 5, 18695, 1301, 4, 24379, 163, 176, 387, 26204, 359, 21013, 573, 1986, 8, 39693, 21013, 1753, 4, 39838, 6, 39693, 6, 8859, 8, 5271, 14249, 1970, 1589, 8712, 1023, 1942, 21013, 14552, 5924, 4, 14219, 714, 709, 634, 714, 4535, 6, 341, 165, 709, 1905, 7, 323, 19104, 12, 11579, 32209, 6, 23209, 1737, 3611, 634, 26878, 8, 20393, 4535, 6, 1412, 618, 397, 12245, 7, 323, 3044, 31070, 8, 2171, 5740, 11, 931, 323, 36, 3976, 8009, 6, 936, 8, 464, 1052, 43, 13, 5, 11199, 573, 2837, 4, 16998, 15, 12, 16395, 322, 10310, 2539, 19, 1778, 7791, 149, 21013, 1753, 8, 21013, 2010, 4, 29474, 2643, 19, 2539, 6, 716, 15, 138, 2820, 8, 275, 3464, 4, 1702, 2655, 19, 4025, 15, 806, 9352, 6, 2643, 6, 275, 3464, 15, 13323, 2167, 8, 67, 15, 14569, 573, 6411, 5441, 37878, 8, 3780, 19468, 6, 6401, 6, 1986, 8, 6196, 13, 4249, 7193, 25, 157, 2111, 12, 16891, 3165, 14877, 4, 653, 47, 836, 35, 497, 513, 195, 2744, 107, 9, 676, 3779, 2010, 9466, 14360, 497, 513, 132, 107, 9, 676, 11, 323, 9, 14249, 1970, 21013, 14552, 24450, 1743, 497, 513, 155, 107, 9, 676, 11, 6494, 573, 11170, 19, 10451, 785, 36, 39884, 1970, 21013, 26204, 6, 8712, 1023, 1942, 6, 274, 245, 12, 2336, 448, 43, 12920, 676, 35, 26403, 33582, 11, 14249, 1970, 21013, 14552, 24450, 16237, 13, 1732, 262, 4, 406, 4, 1178, 7698, 33, 1420, 15, 27831, 15, 14249, 1970, 21013, 14552, 24450, 6275, 11523, 6, 21013, 4827, 6, 21013, 31802, 6, 21013, 17909, 4, 7698, 33, 2655, 15, 40746, 6, 13910, 591, 6494, 32537, 14360, 7, 1045, 39474, 716, 15, 414, 33819, 6, 33000, 27549, 6, 2069, 8, 1263, 10012, 6, 6442, 21247, 523, 15528, 73, 14178, 8458, 154, 6, 43136, 5967, 1527, 6, 6731, 2748, 19, 8503, 36, 2387, 46608, 43, 4753, 4, 43411, 2655, 15, 35, 9511, 12, 705, 19473, 6, 1833, 605, 12019, 6, 2502, 1833, 605, 12019, 6, 7511, 17832, 19859, 6, 4836, 6, 18695, 1301, 43754, 7698, 33, 1006, 15, 21013, 1521, 6, 21013, 46273, 4, 16236, 5206, 5, 4680, 4937, 19, 21013, 4300, 6, 384, 44298, 132, 4, 288, 36, 5632, 344, 25982, 6, 43508, 3260, 6, 3653, 13801, 6, 35712, 17022, 238, 43257, 6, 17255, 46629, 4, 21013, 14552, 24450, 19104, 12, 11579, 39046, 634, 10065, 8, 35495, 30906, 4, 2497, 447, 2655, 11, 15826, 1737, 4, 34473, 6, 39693, 6, 8, 3616, 14249, 1970, 21013, 14552, 24450, 8, 21013, 4827, 15, 15826, 13282, 4, 8504, 62, 6, 39693, 6, 8, 22016, 35476, 22578, 13, 819, 8, 13677, 4, 15347, 6, 3014, 6, 8, 6292, 10700, 26878, 7, 31399, 8558, 6, 4615, 1902, 5588, 6, 8, 1477, 5903, 5838, 4, 1554, 1580, 10228, 1915, 6, 217, 2749, 62, 25583, 8, 5431, 13, 3783, 9453, 6, 21032, 21429, 6, 8, 21675, 980, 4872, 4, 20538, 15661, 15011, 8, 3616, 9202, 3496, 7, 489, 1743, 2823, 8, 22624, 19, 573, 1986, 4, 5427, 8, 3008, 30034, 1635, 6, 11500, 13, 1337, 14249, 1970, 7952, 6060, 19745, 28672, 19, 9865, 32122, 4, 46475, 2407, 8, 3616, 4872, 8, 19980, 3270, 215, 25, 37996, 415, 12326, 6, 229, 1452, 1113, 6, 9359, 620, 1671, 6, 8, 8655, 13825, 13, 2375, 467, 474, 8, 819, 4872, 4, 24373, 11152, 819, 28927, 6, 16300, 4872, 6, 8, 13656, 41086, 7, 1306, 239, 7265, 8, 5443, 9, 1743, 4, 27246, 33843, 6, 5588, 6, 26878, 6, 8, 467, 1022, 7, 323, 5903, 21420, 8, 9666, 2655, 3565, 4, 7698, 33, 1420, 9, 27831, 15, 14249, 1970, 21013, 14552, 24450, 6275, 11523, 6, 21013, 4827, 6, 21013, 31802, 6, 21013, 17909, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'overflow_to_sample_mapping': [0]})
  # 'input_ids': [[etc.]], 'attention_mask': [[]], 'overflow_to_sample_mapping': [0]
  logger.debug("_______________________________________________________________________")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  logger.debug(f"truncated_ids: {type(truncated_ids)} len={len(truncated_ids)} {truncated_ids}")
  overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  logger.debug(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  logger.debug(f"truncated_text:\n{truncated_text}")
  overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)
  logger.debug(f"overflow_text: {overflow_text}")

  logger.debug(f"original_text :\n{text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  overflow_tokens  = tokenizer.tokenize(overflow_text)

  logger.debug(f"original_tokens count : {len(original_tokens)}")
  logger.debug(f"truncated_tokens count: {len(truncated_tokens)}")
  logger.debug(f"overflow_tokens count:  {len(overflow_tokens)}")


In [ ]:
example_text = dataset_dict['train'][0]['text']
get_truncated_part(example_text)

#raise Exception("HHHH")

#### 2) forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database) on dataset_dict['train'][0]['text']: inputs, outputs, logits, probs

##### inputs
No attribute 'shape'.

In [ ]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

logger.debug(f"inputs:{type(inputs)} len={len(inputs)}\n{inputs.keys()}\n{inputs}")
# <class 'transformers.tokenization_utils_base.BatchEncoding'> len=2
# KeysView({'input_ids': tensor([[    0,  1121, etc.,  1, 1]]), 'attention_mask': tensor([[1, 1, etc., 0, 0]])})
# {'input_ids': tensor([[    0,  1121, etc., 1,     1]]), 'attention_mask': tensor([[1, 1, etc., 0, 0]])}


##### outputs
'LongformerSequenceClassifierOutput' object has no attribute 'shape'

In [ ]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
)

logger.debug(f"outputs: {type(outputs)} len={len(outputs)}\nkeys={outputs.keys()}\n{outputs}")
# <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> len=1
# keys=odict_keys(['logits'])
# LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.0412, -0.0641, etc., 0.0711, 0.0780]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


Initializing global attention on CLS token...


##### logits (raw model outputs)
logits are around 0: normal for a model thay is not yet trained

In [ ]:
logits = outputs.logits

logger.debug(f"logits: {type(logits)} len={len(logits)} shape={logits.shape}\n{logits}")
# <class 'torch.Tensor'> len=1 shape=torch.Size([1, 81])
# tensor([[ 0.0412, -0.0641, etc., 0.0711, 0.0780]], grad_fn=<AddmmBackward0>)


##### probs
probs are around 0.5 i.e. for a given label (class) the model considers both possibilites (present or not) equally likely: normal for a model that is not yet trained: logits around 0 => probs around 0.5



In [ ]:
# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)

logger.debug(f"probs:\n{type(probs)}\n{probs.shape}\n{probs}")
# <class 'torch.Tensor'>\torch.Size([1, 81])
# tensor([[0.5103, 0.4840, etc., 0.5178, 0.5195]], grad_fn=<SigmoidBackward0>)

#raise Exception("IIII")


### firstly, example_text, then, example
- example_text = dataset_dict['train'][0]['text']
- example = encoded_dataset['train'][0]

In [ ]:
example = encoded_dataset['train'][0]
#for k, v in example.items():
#    if isinstance(v, np.ndarray):
#        example[k] = v.tolist() # or v.astype(dtype) depending on your desired dtype

logger.debug(f"example: {type(example)} len={len(example)} {example.keys()}\n{example}")
logger.debug("__________________________________________________________________")
logger.debug(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
logger.debug(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
logger.debug(f"example['global_attention_mask']: {type(example['global_attention_mask'])} {len(example['global_attention_mask'])}\n{example['global_attention_mask']}")
logger.debug(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

#raise Exception("JJJJ")


In [ ]:
tokenizer.decode(example['input_ids'])


"<s>Innova - API GateWay Engineer - B2B   Axway API GW, Apigee F5-ASM, REST   Innova   Our client is a prominent player in the financial sector, recognized for its comprehensive range of banking and financial services. With a global presence, it serves a diverse clientele, including individuals, corporations, and institutions. Emphasizing innovation and customer-centricity, it leverages advanced technologies to deliver tailored solutions that meet the evolving needs of its stakeholders. As an API GateWay Engineer (B2B), within Agile Production services Tribe will work together with the existing web squads for the technical design, installation, set-up, industrialization, management, support and documentation of the client's API Infrastructure. You will develop and maintain GW security policies for multiple customer centres within an efficient Agile SDLC for the API Management platform. You will work cross-functionally with Architects, Engineers, and Business Analysts, across multiple t

In [ ]:
labels_list = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
logger.debug(f"labels_list: {type(labels_list)} len={len(labels_list)}\n{labels_list}")

labels_list_key_types = {type(label) for label in labels_list}
logger.debug(f"labels_list_key_types: {labels_list_key_types}")

#raise Exception("KKKK")


In [ ]:
logger.debug(f"input_ids[0]: {type(train_dataset['input_ids'][0])}\tlen={len(train_dataset['input_ids'][0])}\tshape={train_dataset['input_ids'][0].shape}\n{train_dataset['input_ids'][0]}")
logger.debug(f"attention_mask[0]: {type(train_dataset['attention_mask'][0])}\tlen={len(train_dataset['attention_mask'][0])}\tshape={train_dataset['attention_mask'][0].shape}\n{train_dataset['attention_mask'][0]}")
logger.debug(f"global_attention_mask[0]: {type(train_dataset['global_attention_mask'][0])}\tlen={len(train_dataset['global_attention_mask'][0])}\tshape={train_dataset['global_attention_mask'][0].shape}\n{train_dataset['global_attention_mask'][0]}")
logger.debug(f"labels[0]: {type(train_dataset['labels'][0])}\tlen={len(train_dataset['labels'][0])}\tshape={train_dataset['labels'][0].shape}\n{train_dataset['labels'][0]}")
#input_ids[0]: <class 'torch.Tensor'>	len=1024	shape=torch.Size([1024])
#tensor([    0,  1121, 38823,  ...,     1,     1,     1])
#attention_mask[0]: <class 'torch.Tensor'>	len=1024	shape=torch.Size([1024])
#tensor([1, 1, 1,  ..., 0, 0, 0])
#global_attention_mask[0]: <class 'torch.Tensor'>	len=1024	shape=torch.Size([1024])
#tensor([1, 0, 0,  ..., 0, 0, 0])
#labels[0]: <class 'torch.Tensor'>	len=81	shape=torch.Size([81])
#tensor([0., 0., etc., 1., etc, 0., 0.])

#raise Exception("LLLL")


### Explain the following instruction
The selected code snippet performs a forward pass through your LongformerForSequenceClassification model with a single example from your training dataset.

Here's a breakdown:

outputs = model(...): This line calls your model object (which is an instance of LongformerForSequenceClassification) and passes input data to it. This is the core of the model's inference process.

- input_ids = train_dataset['input_ids'][0].unsqueeze(0): This accesses the input_ids tensor for the first example ([0]) in your training dataset.
  - .unsqueeze(0): Models typically expect inputs in batches, even if there's only one example. unsqueeze(0) adds a new dimension at the beginning of the tensor, effectively creating a batch of size 1.
- attention_mask = train_dataset['attention_mask'][0].unsqueeze(0): Similar to input_ids, this accesses the attention_mask for the first example and adds a batch dimension. The attention mask tells the model which tokens are actual input and which are padding.
- labels = train_dataset[0]['labels'].unsqueeze(0): This accesses the ground truth labels for the first example and adds a batch dimension.
  - Providing labels during the forward pass allows the model to calculate the loss, which is useful for debugging or understanding how the model is performing on a specific example.

In essence, this code is feeding one data point (the first example from the training set, including its text tokenized as input_ids and attention_mask, and its corresponding labels) into the model to get the model's output (outputs). Since labels are provided, the outputs object will likely include the calculated loss for this single example.


Is unsqueeze(0) required?

That's a great observation! You didn't see unsqueeze(0) in the cells where you were processing the entire dataset_dict or using the trainer.

The reason it's used in the selected cell is because you are explicitly taking a single example from the train_dataset ([0]) and passing it directly to the model.

Here's the difference:

When processing the whole dataset or using the Trainer: the Hugging Face datasets library and the Trainer are designed to handle data in batches. When you iterate through a dataset or pass it to the Trainer, the library automatically groups the examples into batches and provides them to the model with the correct batch dimension already present. You don't need to manually add it.

When processing a single example manually: If you want to feed just one example to the model outside of the batching mechanism provided by datasets or Trainer (like in the selected cell for demonstration or debugging), you need to manually add the batch dimension using .unsqueeze(0). This is because most models are designed to process data in batches, even if the batch size is one. The model expects input tensors of shape (batch_size, sequence_length) or (batch_size, num_labels), not just (sequence_length) or (num_labels).
So, while unsqueeze(0) is not needed when the data is already batched (by datasets or Trainer), it is required when you are working with a single example tensor and feeding it directly to a model that expects batched input.



In [ ]:
outputs = model(
    input_ids      = train_dataset['input_ids'][0].unsqueeze(0),
    attention_mask = train_dataset['attention_mask'][0].unsqueeze(0),
    labels         = train_dataset['labels'][0].unsqueeze(0)
)

# 'LongformerSequenceClassifierOutput' object has no attribute 'shape'
logger.debug(f"outputs: {type(outputs)} {outputs.keys()} len={len(outputs)}\n{outputs}")
#<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['loss', 'logits']) len=2
#LongformerSequenceClassifierOutput(loss=tensor(0.6859, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
#logits=tensor([[ 0.0490, -0.0186, etc., -0.0089, 0.0407]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)

#raise Exception("MMMM")


## Start training, evaluation and test

- 3 steps: training (train_dataset), evaluation (validation_dataset), test (test_dataset)
- 3 Trainer functions: train, evaluate, predict


### Training


#### Weighted loss function
**weight and pos_weight**:
torch.nn.BCEWithLogitsLoss function is a commonly used loss function for binary classification problems, where model output is a probability value between 0 and 1. It combines a sigmoid activation function with a binary cross-entropy loss.

For imbalanced datasets, where number of a class is significantly smaller than other, BCEWithLogitsLoss can be modified by adding a _weight parameter_ to loss function.

BCEWithLogitsLoss also has a _pos_weight parameter_, which is a simpler way to specify weight for positive class (equivalent to weight parameter = [ 1, pos_weight], where weight for negative class = 1.

Negative samples (0s) are not weighted explicitly because the loss function already balances them implicitly.

pos_weights stands for positive weights in the BCEWithLogitsLoss function.

In multi-label classification, each label is a separate binary classification problem.

Each label has positive sample (1s) and negative samples (0s) in the dataset.
- If a label is rare (fewer 1s), its weight will be higher -> encourages the model to predict it more often
- If a label is common (many 1s), its weight will be lower -> prevents the model to overpredicting it

**Normalization**:
Without normalization, pos_weights might have huge variations across labels, that could destabilize training.
- Min-Max Scaling:
    - Rescales values between 0 and 1
    - Reduces large variations but keeps relative ranking
- Z-Score Normalization:
    - Centers values around 0 with a standard deviation of 1
    - Handles outliers better than min-max
- Sum-to-One Scaling:
    - Makes weights sum to 1, preventing extremely large values

Recommended approach: try sum-to-one normalization first. If performance is unstable, test z-score.

#### Weighted BCEWithLogitsLoss
Assigns higher weights to rare labels using class weights.

In [ ]:
def class_weights(labels):
    logger.debug(f"labels: {type(labels)} len={len(labels)}\n{labels}")

    if isinstance(labels, datasets.arrow_dataset.Column):
        labels = np.array(labels)
    logger.debug(f"labels: {type(labels)} len={len(labels)} shape={labels[:].shape}\n{labels}")

    num_samples = num_labels = 0
    if labels.ndim == 1:
        num_samples = labels.shape[0]
        num_labels  = 1
    else:
        num_samples, num_labels = labels.shape

    logger.debug(f"num_samples: {type(num_samples)} {num_samples}")
    logger.debug(f"num_labels:  {type(num_labels)}  {num_labels}")

    # class_counts = how many times each label appears (i.e. number of 1s per label)
    # (dim=0 means summing across all samples; equivalent to axis = 0 for Pandas DataFrame)
    class_counts = labels.sum(axis=0)
    logger.debug(f"class_counts: {type(class_counts)} len={len(class_counts)}\n{class_counts}")

    # pos_weights = negative samples (0s) per label / positive samples (1s) per label
    pos_weights = (num_samples-class_counts) / (class_counts + 1e-6)  # Avoid division by zero
    logger.debug(f"pos_weights: {type(pos_weights)} len={len(pos_weights)}\n{pos_weights}")

    # Normalization
    normalized_pos_weights_minmax = (pos_weights-pos_weights.min()) / (pos_weights.max()-pos_weights.min())
    logger.debug(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

    normalized_pos_weights_zscore = (pos_weights-pos_weights.mean()) / pos_weights.std()
    logger.debug(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

    normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
    logger.debug(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

    #return pos_weights
    #return normalized_pos_weights_minmax
    #return normalized_pos_weights_zscore
    return normalized_pos_weights_sum1


In [ ]:
pos_weights        = class_weights(train_dataset['labels'])
pos_weights_tensor = torch.tensor(pos_weights, dtype=torch.float).to(device)

bce_loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights_tensor)

#raise Exception("NNNN")

#### Focal Loss
Reduces the impact of easy samples (majority class) and focuses on difficult samples (minority class).

Standard Binary Cross-Entropy (BCE) gets dominated by easy negative samples.

- α (alpha): Adjusts class weighting (balancing positive (1) and negative (0) samples.
  - α < 0.5: down-weights negatives (majority)
  - α > 0.5: up-weights positive (minority)
- γ (gamma): Controls how much hard-to-classify samples are emphasized.
  - γ = 0: no effect -> reduces to normal BCE
  - γ = 1: Some focus on hard samples
  - γ = 2: Stronger focus on hard samples
  - γ > 2: Extreme focus on hard samples (can underfit esady samples)

In [ ]:
class FocalLoss(Module):
    """
    Focal Loss implementation for handling class imbalance.
    """
    counter = 0

    def __init__(self, alpha=1.0, gamma=2.0, logits=True, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha     = alpha
        self.gamma     = gamma
        self.logits    = logits     # True if inputs are logits, False if probabilies
        self.reduction = reduction  # 'mean' or 'none'

        FocalLoss.counter += 1

    # inputs  = model's predictions: PyTorch tensor, shape=(batch_size, num_classes)
    # targets = ground truth labels: PyTorch tensor, shape=same as inputs shape
    def forward(self, inputs, targets):
        targets = targets.to(inputs.device)  # Ensure labels are on the same device

        #logger.debug(f"Focal Loss: def forward: inputs: {type(inputs)}\ntargets: {type(targets)}")
        # Check whether input is probability or logits
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')

        pt         = torch.exp(-BCE_loss)  # Probability of the correct class
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        return focal_loss.mean() if self.reduction == 'mean' else focal_loss

    def __repr__(self):
        return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduction={self.reduction})"

    def __str__(self):
        return self.__repr__()


In [ ]:
#focal_loss_fn = FocalLoss(alpha=0.5, gamma=3.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.25, gamma=4.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.625, gamma=4.0, logits=True, reduction='mean')

focal_loss_fn = FocalLoss(alpha=alpha, gamma=gamma, logits=True, reduction='mean')

logger.debug(f"focal_loss_fn: {type(focal_loss_fn)} {focal_loss_fn}")


In [ ]:
class LossLoggerCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Retrieve loss from logs
        logs       = kwargs.get("logs", {})
        epoch_loss = logs.get("logs", None)  # Loss from the Trainer logs

        if epoch_loss is not None:
            logging.info(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
            #logger.debug(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
        else:
            logging.warning(f"Epoch {state.epoch:.0f} - No loss logged!")
            #logger.debug(f"Epoch {state.epoch:.0f} - No loss logged!")


In [ ]:
class MetricsLoggerCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None:
            metrics = kwargs.get("metrics")  # Ensure we get the metrics if passed in kwargs
        if metrics:                          # Check if metrics exist
            logging.info(f"Epoch {state.epoch:.0f} - "
                         f"Precision: {metrics.get('precision', float('nan')):.4f} - "
                         f"Recall: {metrics.get('recall', float('nan')):.4f} - "
                         f"F1: {metrics.get('f1', float('nan')):.4f}")


In [ ]:
class ProgressLoggerCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        logs       = kwargs.get("logs", {})  # Extract from logs
        epoch_loss = logs.get("loss", None)  # Get loss value

        if epoch_loss is not None:
            logging.info(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
            #logger.debug(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
        else:
            logging.info(f"Epoch {state.epoch:.0f} - No loss logged!")
            #logger.debug(f"Epoch {state.epoch:.0f} - No loss logged!")

    def on_train_end(self, args, state, control, **kwargs):
        logging.info("=== Training Completed! ===")
        #logger.debug("=== Training Completed! ===")


#### Training Metrics
  source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

#### UndefinedMetricWarning
Only one class is present in y_true. ROC AUC score is not defined in that case.

This warning typically arises when you're trying to calculate the ROC AUC score for a label where either all true values are 0 or all are 1 in a particular batch.

The ROC AUC score is calculated by comparing the true positive rate (TPR) against the false positive rate (FPR) at various thresholds. If a label is only ever predicted as 0 or 1, you cannot generate a meaningful ROC curve and thus the AUC is undefined.

AUC scores rely on the presence of both positive and negative samples for each label.

Solution:

- Check Label Distribution: Add a check at the start to see if either true_labels or preds for a particular label contain only one unique value (0 or 1). If so, for that label, either skip the ROC AUC calculation or set the ROC AUC to a default value (like 0 or NaN).
- Ignore the warning (not recommended):
- **Stratified sampling: While you did split into train/validation/test, the warning may indicate you didn't maintain the label balance during the splitting process. Stratified sampling would do this.**

#### TPR (True Positive Rate) and FPR (False Positive Rate)
TPR = Sensitivity = Recall = TP / (TP + FN)
- TPR close to 1: the model identifies most positives
- TPR close to 0: the model is missing many positives

FPR = FP / (FP + TN)
- FPR close to 1: the model produces many false alarms
- FPR close to 0: the model makes few false alarms

The ROC curve plots TPR (y-axis) vs. FPR (x-axis) at different thresholds.
A perfect model has:
- TPR = 1 (detect all positives)
- FPR = 0 (no false alarms)

The ideal ROC curve is a steep rise towards the top-left corner.

#### zero_division=0

- only for f1, precision and recall because they involve division where the denominator can be zero: some labels might never       be predicted (y_pred = 0 for all samples), or they might not appear in the true_labels (y_true = 0 for all samples)
- ROC AUC: works with probabilities and does not involve division by zero
- Precision-Recall AUC: also based on ranking, so no zero division issue
- Accuracy: just compares exact matches, so no zero division issue

In [ ]:
def multi_label_metrics(logits, true_labels, threshold=0.5, average='micro'):
    """
    Compute multi-label classification metrics.

    Params:
    - logits      (np.ndarray or torch.Tensor): raw, unnormalized model outputs, shape (n_samples, n_labels)
    - true_labels (np.ndarray)                : ground truth binary labels shape (n_samples, n_labels)
    - threshold   (float or np.ndarray)       : 0.5 (default)
    - average     (str)                       : 'micro' (default); no per-label scores i.e. not average=None

    Returns:
    - metrics (dict)
    """
    global multi_label_metrics_counter
    # The flush=True argument ensures that the message is immediately printed to the console without being buffered.
    # This is helpful for debugging to see the exact point in the program execution where the function is called initially.
    if multi_label_metrics_counter == 0:
        logger.debug(">>>>>>>>>>multi_label_metrics entered for the first time<<<<<<<<<<")
    multi_label_metrics_counter += 1

    logger.debug(f"multi_label_metrics: logits: {type(logits)} shape={logits.shape}\n{logits}")
    # <class 'numpy.ndarray'> (5000, 147) <<<<<<<<<<< <class 'numpy.ndarray'>, and not <class 'torch.Tensor'>
    logger.debug(f"multi_label_metrics: true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")
    # <class 'numpy.ndarray'> (5000, 147)

    # Handle both float and array thresholds
    if isinstance(threshold, (int, float)):
        logger.debug(f"multi_label_metrics: threshold: {type(threshold)} {threshold}")
        # <class 'float'> 0.5
    else:
        logger.debug(f"multi_label_metrics: threshold: {type(threshold)} shape={threshold.shape}\n{threshold}")

    # Ensure logits are a PyTorch tensor
    if isinstance(logits, np.ndarray):
        logits = torch.from_numpy(logits)
    logger.debug(f"multi_label_metrics: logits: {type(logits)} shape={logits.shape}\n{logits}")
    # <class 'torch.Tensor'> shape=torch.Size([5000, 147])
    # of course, same values as numpy.ndarray

  # Apply sigmoid to get probabilities; convert to NumPy safely:
  # - detach() remove the tensor from the computation graph, making it a regular tensor without gradients
  # - cpu() moves the tensor from the GPU to the CPU before converting to numpy.ndarray
    probs = torch.sigmoid(logits).detach().cpu().numpy()
    logger.debug(f"multi_label_metrics: probs: {type(probs)} shape={probs.shape}\n{probs}")
    # <class 'numpy.ndarray'> shape=(5000, 147)

    # Apply threshold to get binary predictions
    # NOTE: probs is a np.ndarray, NumPy can compare a np.ndarray to
    # - a scalar (float/int/NumPy scalar), another np.ndarray of the same shape,
    # - but not to a PyTorch tensor
    preds = (probs > threshold).astype(int)
    logger.debug(f"multi_label_metrics: preds: {type(preds)} shape={preds.shape}\n{preds}")
    # <class 'numpy.ndarray'> shape=(5000, 147)

    # Compute core metrics
    # Use zero_division='warn' or 0 or 1 explicitly
    # zero_division='warn' is the default and recommended approach
    # zero_division=0 treats zero division as resulting in 0
    # zero_division=1 treats zero division as resulting in 1
    f1        = f1_score       (true_labels, preds, average=average, zero_division='warn')
    precision = precision_score(true_labels, preds, average=average, zero_division='warn')
    recall    = recall_score   (true_labels, preds, average=average, zero_division='warn')

    # To avoid computing AUC scores on labels that are always 0 or always 1;
    # identify valid labels (those with both 0s and 1s in 'y_true' i.e. those present in some but not all samples)
    valid_labels = np.where(
        (true_labels.sum(axis=0) > 0) & (true_labels.sum(axis=0) < true_labels.shape[0])
    )[0]
    logger.debug(f"multi_label_metrics: valid_labels: {type(valid_labels)} len={len(valid_labels)}\n{valid_labels}")
    # valid_labels: <class 'numpy.ndarray'> len=142
    # [ 0  1  etc.  44  46  47  49  50  53  55  etc. 145 146]
    # invalid labels: 45: '903', 48: '907', 51: '911', 52: '912', 54: '914'

    # Curve-based metrics (mean across valid labels). These are label-wise, then averaged over valid labels.
    if len(valid_labels) > 0:
        roc_auc = np.mean([
            roc_auc_score(true_labels[:, i], probs[:, i])
            for i in valid_labels  # valid_labels = indices
        ])
        pr_recall_auc = np.mean([
            average_precision_score(true_labels[:, i], probs[:, i])
            for i in valid_labels
        ])
    else:
        roc_auc       = np.nan
        pr_recall_auc = np.nan

    # Other useful metrics:
    # - subset accuracy : % of samples with all labels correctly predicted, exact match between all predicted and true labels per sample
    #                   : = total OK / total KO of my Prediction global score
    # - hamming loss    : % of misclassified labels: counts any mismatch (FP and FN), ignores correct matches (TP and TN)
    # - hamming accuracy: average number of correctly predicted labels (0s and 1s) across all samples and labels
    subset_acc  = accuracy_score(true_labels, preds)
    hamming_los = hamming_loss(true_labels, preds)
    hamming_acc = 1 - hamming_loss(true_labels, preds)

    return {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': pr_recall_auc,
        'subset_accuracy'     : subset_acc,
        'hamming_loss'        : hamming_los,
        'hamming_accuracy'    : hamming_acc
    }


In [ ]:
# Usage with HF Trainer
def compute_metrics(p: EvalPrediction):
    global compute_metrics_counter
    if compute_metrics_counter == 0:
        logger.debug(">>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<")
    compute_metrics_counter += 1

    logger.debug(f"p.predictions: {type(p.predictions)} len={len(p.predictions)}\n{p.predictions}")
    # p.predictions: <class 'numpy.ndarray'> len=5000
    logger.debug(f"p.label_ids: {type(p.label_ids)} len={len(p.label_ids)}\n{p.label_ids}")
    # p.label_ids: <class 'numpy.ndarray'> len=5000
    if isinstance(threshold, (int, float)):
        logger.debug(f"threshold: {type(threshold)} {threshold}")
        # threshold: <class 'float'> 0.5
    else:
        logger.debug(f"threshold: {type(threshold)} shape={threshold.shape}\n{threshold}")


    preds  = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

    return multi_label_metrics(preds, p.label_ids, threshold=threshold, average=training_average)


#### HF transformer Trainer and CustomTrainer
Abstracts the training loop.

In [ ]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',               # Evaluate once per epoch
    save_strategy               = 'epoch',               # Save checkpoints once per epoch
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = 'f1',
    greater_is_better           = True,
    fp16                        = True,
    #run_name                    = run_name,
    report_to                   = 'none'                 # Disable wandb
)


In [ ]:
class CustomTrainer(Trainer):
    compute_loss_counter = 0

    def __init__(self, model, *args, loss_fn=None, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.loss_fn = loss_fn
        logger.debug(f">>>>>>>>>>CustomTrainer initialized with loss_fn: {loss_fn}<<<<<<<<<<")

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        if CustomTrainer.compute_loss_counter == 0:
            logger.debug(f">>>>>>>>>>CustomTrainer: compute_loss entered for the first time<<<<<<<<<<")
        CustomTrainer.compute_loss_counter += 1

        labels  = inputs.get('labels')
        outputs = model(**inputs)
        logits  = outputs.logits  # (batch_size, num_labels)

        if labels is not None:
            labels = labels.to(logits.device).float()  # Ensure same device

            if self.loss_fn is not None:
                loss = self.loss_fn(logits, labels)
                #logging.info(f"Step Loss ({self.loss_fn.__class__.__name__}): {loss.item():.6f}")  # Log loss value
                logger.debug(f"Epoch {self.state.epoch:.0f}, Step {self.state.global_step}: Loss ({self.loss_fn.__class__.__name__}): {loss.item():.6f}")
                # Epoch 0, Step 0: Loss (FocalLoss): 0.251234
            else:
                loss_fn = BCEWithLogitsLoss()
                loss    = loss_fn(logits, labels)
                #logging.info(f"Step Loss (BCEWithLogitsLoss): {loss.item():.6f}")
                logger.debug(f"Epoch {self.state.epoch:.0f}, Step {self.state.global_step}: Loss (BCEWithLogitsLoss): {loss.item():.6f}")

            return (loss, outputs) if return_outputs else loss

        return outputs

    def get_CustomTrainer_compute_loss_counter(self):
        return CustomTrainer.compute_loss_counter


In [ ]:
trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_dataset,
    eval_dataset    = validation_dataset,

    compute_metrics = compute_metrics,
    loss_fn         = focal_loss_fn,  # bce_loss_fn or focal_loss_fn
    #callbacks       = [LossLoggerCallback(), MetricsLoggerCallback(), ProgressLoggerCallback()]  # Attach logging callbacks
)

#trainer = Trainer(
#    model           = model,
#    args            = training_args,
#    train_dataset   = train_dataset,
#    eval_dataset    = validation_dataset,

#    compute_metrics = compute_metrics,
#)

logger.debug(f"trainer: {type(trainer)} {trainer}")


#### trainer.train

In [ ]:
trainer_train = trainer.train()

logger.debug(f"trainer_train: {type(trainer_train)} len={len(trainer_train)}\n{trainer_train}")
# trainer_train: <class 'transformers.trainer_utils.TrainOutput'> len=3
# TrainOutput(global_step=1250, training_loss=0.04169110412597656, metrics={'train_runtime': 2051.2884, 'train_samples_per_second': 19.5, 'train_steps_per_second': 0.609, 'total_flos': 2.630146793472e+16, 'train_loss': 0.04169110412597656, 'epoch': 1.0})

logger.debug("__________________________________________________________________")
trainer_train_metrics        = trainer_train.metrics
trainer_train_metrics_native = convert_to_native(trainer_train_metrics)

#logger.debug(f"trainer.train: trainer_train_metrics: {type(trainer_train_metrics)} len={len(trainer_train_metrics)}\n{trainer_train_metrics}")
# <class 'dict'> len=6
# {'train_runtime': 7.3337, 'train_samples_per_second': 7.909, etc.
#logger.debug(f"trainer.train: trainer_train_metrics: {type(trainer_train_metrics)} len={len(trainer_train_metrics)}\n{json.dumps(trainer_train_metrics, indent=4)}")
# <class 'dict'> len=6
# as above but indented
logger.debug(f"trainer.train: trainer_train_metrics_native: {type(trainer_train_metrics_native)} len={len(trainer_train_metrics_native)}\n{trainer_train_metrics_native}")
# <class 'dict'> len=6
# {'train_runtime': 7.3337, 'train_samples_per_second': 7.909, etc.
#logger.debug(f"trainer.train: trainer_train_metrics_native: {type(trainer_train_metrics_native)} len={len(trainer_train_metrics_native)}\n{json.dumps(trainer_train_metrics_native, indent=4)}")
# trainer_train.metrics: <class 'dict'> len=6
# as above but indented

logger.debug(f"after trainer.train(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
logger.debug(f"after trainer.train(): compute_metrics_counter           : {compute_metrics_counter}")
logger.debug(f"after trainer.train(): compute_loss_counter              : {compute_loss_counter}")
logger.debug(f"after trainer.train(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
logger.debug(f"after trainer.train(): FocalLoss.counter                 : {FocalLoss.counter}")

#raise Exception("OOOO")


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Subset Accuracy,Hamming Loss,Hamming Accuracy,Runtime,Samples Per Second,Steps Per Second
1,No log,0.244723,0.055172,0.030303,0.307692,0.538095,0.490363,0.000000,0.483245,0.516755,0.209300,33.437000,4.777000


In [ ]:
logger.info("trainer.train successfully completed")


[INFO] trainer.train successfully completed


#### Like the above forward pass for debugging or verification purposes on dataset_dict['train'][0]['text'], for comparison,: recalculate inputs, outputs, logits, probs after training.

In [ ]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

logger.debug(f"inputs: {type(inputs)} len={len(inputs)}\n{inputs.keys()}\n{inputs}")
# inputs: <class 'transformers.tokenization_utils_base.BatchEncoding'> len=2
# KeysView({'input_ids': tensor([[    0,  1121, 38823,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])})
logger.debug(f"inputs['input_ids']: {type(inputs['input_ids'])} len={len(inputs['input_ids'])} shape={inputs['input_ids'].shape}\n{inputs['input_ids']}")
# inputs['input_ids']: <class 'torch.Tensor'> len=1 shape=torch.Size([1, 1024])
logger.debug(f"inputs['attention_mask]: {type(inputs['attention_mask'])} len={len(inputs['attention_mask'])} shape={inputs['attention_mask'].shape}\n{inputs['attention_mask']}")
# inputs['attention_mask]: <class 'torch.Tensor'> len=1 shape=torch.Size([1, 1024])
logger.debug(f"device: {device}")
# device: cuda


#### PyTorch tensors are created on the CPU by default.

This was not necessary in the check before training.

The error "RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!" means that your model and input data are on different devices (CPU and GPU). To fix this, you need to ensure both are on the same device.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs.to(device)
logger.debug(f"device: {device}")
# device: cuda


#### 'LongformerSequnceClassifierOuput' object has no attribute 'shape'

In [ ]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
)

logger.debug(f"outputs: {type(outputs)} len={len(outputs)}\nkeys={outputs.keys()}\n{outputs}")
# outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> len=1
# keys=odict_keys(['logits'])


In [ ]:
logits = outputs.logits

logger.debug(f"logits: {type(logits)} len={len(logits)} shape={logits.shape}\n{logits}")
# logits: <class 'torch.Tensor'> len=1 shape=torch.Size([1, 81])


In [ ]:
probs = sigmoid(logits)

logger.debug(f"probs: {type(probs)} len={len(probs)} shape={probs.shape}\n{probs}")
# <class 'torch.Tensor'> len=1 shape=torch.Size([1, 81])

#raise Exception("PPPP")


#### trainer.train: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name               = "trainer_train_native"
    trainer_train_path = f"{name}.json"

    with open(trainer_train_path, "w") as f:
        json.dump(trainer_train, f)

    api.upload_file(
        path_or_fileobj = trainer_train_path,
        path_in_repo    = trainer_train_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} results successfully uploaded to HF Hub as {trainer_train_path}")


[INFO] trainer_train_native results successfully uploaded to HF Hub as trainer_train_native.json


#### trainer.train: check that the uploaded file can be downloaded
File locally downloaded to:
/root/.cache/huggingface/hub/datasets-claudelepere-skill_classification/snapshots/full_commit_hash/trainer_train_results.json

In [ ]:
if ACCESS_TO_HF_HUB:
  file_path = hf_hub_download(repo_type="model", repo_id=repo_id, filename=trainer_train_path)

  logger.debug(f"file_path: {file_path}")
  # file_path: /root/.cache/huggingface/hub/models--claudelepere--jobs_EN_791112_50000_all_2025-07-14/snapshots/981a46c4e9a788893926d1826727acea144de932/trainer_train.json


trainer_train_native.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

### Upload tokenizer and model to HF Hub


In [ ]:
if ACCESS_TO_HF_HUB:

    # Upload the tokenizer
    commit_message = f"tokenizer-{timestamp}"
    tokenizer.push_to_hub(repo_id, commit_message=commit_message)

    # Upload the model
    commit_message = f"model-{timestamp}"
    model.push_to_hub(repo_id, commit_message=commit_message)

    logger.info(f"tokenizer and model successfully uploaded to HF Hub at {repo_id}")
    logger.debug("__________________________________________________________________")

    # Check
    def check_upload(repo_id):
        logger.debug("Tokenizer")
        tokenizer = LongformerTokenizerFast.from_pretrained(repo_id)
        logger.debug("______________________________________________________________")

        logger.debug("Model")
        model = LongformerForSequenceClassification.from_pretrained(repo_id)
        logger.debug("______________________________________________________________")

        inputs  = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        logger.debug(f"outputs: {type(outputs)} len={len(outputs)}\n{outputs.keys()}\n{outputs}")
        # outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> len=1
        # outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>

    # To check whether upload was successful, download the tokenizer and the model
    check_upload(repo_id)


README.md:   0%|          | 0.00/784 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/783 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp6j2dc_b3/model.safetensors    :   2%|1         | 9.35MB /  595MB            

...

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Input ids are automatically padded to be a multiple of `config.attention_window`: 1024


### Evaluation

#### Evaluation 1: trainer.evaluate
trainer.evaluate uses a fixed threshold of 0.5 to convert logits into binary labels, which is often suboptimal for imbalanced data.

In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

# by default, trainer.evaluate() evaluates the dataset passed as eval_dataset during training
# metrics_key_prefix: prefix for the evaluation metrics
evaluation_trainer_evaluate_metrics        = trainer.evaluate(metric_key_prefix="eval")
evaluation_trainer_evaluate_metrics_native = convert_to_native(evaluation_trainer_evaluate_metrics)

logger.debug(f"evaluation_trainer_evaluate_metrics_native: {type(evaluation_trainer_evaluate_metrics_native)} len={len(evaluation_trainer_evaluate_metrics_native)}\n{evaluation_trainer_evaluate_metrics_native}")
# evaluation_trainer_evaluate_metrics: <class 'dict'> len=13
logger.debug("__________________________________________________________________")

logger.debug(f"after trainer.evaluate(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
logger.debug(f"after trainer.evaluate(): compute_metrics_counter           : {compute_metrics_counter}")
logger.debug(f"after trainer.evaluate(): compute_loss_counter              : {compute_loss_counter}")
logger.debug(f"after trainer.evaluate(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
logger.debug(f"after trainer.evaluate(): FocalLoss.counter                 : {FocalLoss.counter}")


In [ ]:
logger.info("evaluation 1: trainer.evaluate: successfully completed")


[INFO] evaluation 1: trainer.evaluate: successfully completed


#### Evaluation 1: trainer.evaluate: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name                             = "evaluation_trainer_evaluate_native"
    evaluation_trainer_evaluate_path = f"{name}.json"

    with open(evaluation_trainer_evaluate_path, "w") as f:
        json.dump(evaluation_trainer_evaluate_metrics_native, f)

    api.upload_file(
        path_or_fileobj = evaluation_trainer_evaluate_path,
        path_in_repo    = evaluation_trainer_evaluate_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_evaluate_path}")


[INFO] evaluation_trainer_evaluate_native successfully uploaded to HF Hub as evaluation_trainer_evaluate_native.json


#### Evaluation 2: trainer.predict

In [ ]:
def exact_match_score(thresholds, probs, true_labels):
    preds = (probs > thresholds).astype(int)
    return -np.mean(np.all(preds == true_labels, axis=1))  # return negative for minimization: minimizing the negative exact match score is equivalent to maximizing the exact match score.


In [ ]:
def optimize_thresholds_for_exact_match(probs, true_labels, bounds=None, seed=42):
    num_labels = probs.shape[1]

    # Default bounds: each threshold ∈ [0.2, 0.6]
    if bounds is None:
        bounds = [(0.2, 0.6)] * num_labels

    result = differential_evolution(
        exact_match_score,
        bounds=bounds,
        args=(probs, true_labels),
        strategy='best1bin',
        maxiter=20,         # keep this low if compute is limited
        popsize=15,
        tol=1e-4,
        mutation=(0.5, 1),
        recombination=0.7,
        seed=seed,
        polish=True,
        disp=True
    )

    best_thresholds = result.x
    best_score = -result.fun
    return best_thresholds, best_score


In [ ]:
def predict_with_optimized_thresholds_SAV(
        trainer,
        dataset,
        id2label,  # {0: '142', 1: '143', 2: '146', 3: '147', 4: '149', 5: '150', etc.
        threshold_tuning = False,
        thresholds       = None
    ):
    """
    Predicts using trainer.predict(), with optional threshold tuning, using NumPy arrays, and not PyTorch tensors.

    Parameters:
    - trainer                         : Hugging Face Trainer or CustomTrainer instance
    - dataset                         : Dataset to predict on
    - id2label                        : Dictionary mapping label indices (int) to label names (str)
    - threshold_tuning                : Boolean to enable threshold tuning per class (i.e. per label)
    - thresholds       (numpy.ndarray): Custom thresholds for classification

    Returns:
    if threshold_tuning:
        - best_thresholds      (numpy.ndarray): optimized threshold per class (label)
        - best_thresholds_dict (dict)         : optimized threshold per class (label)
        - best_metrics         (dict)         : best F1, best precision, best recall per class (label)
        - best_preds           (numpy.ndarray): best predictions per class (label)
    else:
        - thresholds      (numpy.ndarray): fixed threshold per class (label)
        - thresholds_dict (dict)         : fixed threshold per class (label)
        - metrics         (dict)         : computed with provided thresholds or default to 0.5
        - preds           (numpy.ndarray): predictions with provided thresholds or default to 0.5
    """
    # Predict
    predictions_output = trainer.predict(dataset)
    logits             = predictions_output.predictions # raw predictions made by the model
    true_labels        = predictions_output.label_ids   # ground truth labels
    metrics            = predictions_output.metrics     # metrics computed during evaluation (NOT USED)

    # Sigmoid function: convert logits to probabilities (with np, not with torch)
    probs = 1 / (1 + np.exp(-logits))

    logger.debug(f"predict_with_optimized_thresholds: id2label          : {type(id2label)} len={len(id2label)}\n{id2label}")
    # <class 'dict'> len=147 {0: '142', 1: '143', etc. : order of labels = by SkillTypeIdComplexityId
    logger.debug(f"predict_with_optimized_thresholds: predictions_output: {type(predictions_output)} len={len(predictions_output)}")
    # <class 'transformers.trainer_utils.PredictionOutput'> len=3: tuple (predictions_output.predictions / label_ids /metrics)
    logger.debug(f"predict_with_optimized_thresholds: logits            : {type(logits)} shape={logits.shape}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: true_labels       : {type(true_labels)} shape={true_labels.shape}\n{true_labels}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: probs             : {type(probs)} shape={probs.shape}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: metrics           : {type(metrics)} len={len(metrics)}\n{metrics}")
    # <class 'dict'> len=12
    # {'test_loss': 0.2430032342672348, 'test_f1': 0.11612903225806452, 'test_precision': 0.06338028169014084, 'test_recall': 0.6923076923076923, 'test_roc_auc': 0.5492063492063493, 'test_precision_recall_auc': 0.4570294784580498, 'test_subset_accuracy': 0.0, 'test_hamming_loss': 0.48324514991181655, 'test_hamming_accuracy': 0.5167548500881834, 'test_runtime': 0.1912, 'test_samples_per_second': 36.62, 'test_steps_per_second': 5.231}

    num_labels       = probs.shape[1]  # = number of columns = number of labels = len(id2label) = 147
    best_thresholds  = best_thresholds_dict  = best_metrics  = best_preds  = None
    thresholds_fixed = thresholds_fixed_dict = metrics_fixed = preds_fixed = None

    if threshold_tuning:
        threshold_candidates = np.linspace(0.05, 0.95, 19)
        best_thresholds      = np.zeros(num_labels)
        # best_metrics keys as int <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        best_metrics         = {int(label): {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 0} for label in id2label.values()}
        logger.debug(f"predict_with_optimized_thresholds: threshold_candidates: {type(threshold_candidates)} shape={threshold_candidates.shape}\n{threshold_candidates}")
        # <class 'numpy.ndarray'> shape=(19,)
        logger.debug(f"predict_with_optimized_thresholds: best_thresholds     : {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
        # <class 'numpy.ndarray'> shape=(147,) [0. 0. etc.
        logger.debug(f"predict_with_optimized_thresholds: best_metrics        : {type(best_metrics)} len={len(best_metrics)}\n{best_metrics}")
        # <class 'dict'> len=147 {142: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0}, etc.
        logger.debug("__________________________________________________________________")

        # Iterate over each label to find the best threshold
        # label_idx: model index
        for label_idx, label_str in id2label.items():
            # Predictions for the current label across all threshold candidates: preds shape=(num_samples, num_thresholds)
            # Create preds: an inner loop (across 19 threshold_candidates) vectorized via NumPy,
            # within the outer loop (across labels)

            # [:, label_idx]: 1D array shape=(num_samples, ) = tuple that starts always with the size of the first dimension, and size of the second dimension (if it exists)
            # [:, None]: NumPy's way of adding a new axis, converting a 1D array into a column vector (= a 2D array with a single column shape=(num_samples, 1))
            # probs[:, label_idx][:, None] > threshold_candidates shape=(num_samples, num_thresholds) thanks to NumPy Broadcasting
            preds = probs[:, label_idx][:, None] > threshold_candidates

            logger.debug(f"predict_with_optimized_thresholds: probs[:, label_idx][:, None]: {type(probs[:, label_idx][:, None])} shape={probs[:, label_idx][:, None].shape}")
            # <class 'numpy.ndarray'> shape=(5000, 1)
            logger.debug(f"predict_with_optimized_thresholds: preds: {type(preds)} shape={preds.shape}")
            # <class 'numpy.ndarray'> shape=(5000, 19)

            # Compute precision, recall, F1, support for all thresholds at once for the current label
            # _______________________________________________________________________________________
            # true_labels[:, label_idx]: This selects all rows (:) for a specific label_idx
            # from the true_labels NumPy array. This results in a 1-dimensional NumPy array containing
            # the true binary labels (0 or 1) for that single label across all samples.

            # np.tile(..., (len(threshold_candidates), 1)): This function replicates the column vector
            # shape=(num_samples, 1) created in the previous step.
            # It's tiled len(threshold_candidates) times along the first dimension (rows) and
            # 1 time along the second dimension (columns).
            # This creates a 2D array where each column is the original 1D array of true labels
            # for the current label_idx. The shape becomes (num_samples, len(threshold_candidates)).

            # .T: This transposes the resulting 2D array.
            # The shape changes from (num_samples, len(threshold_candidates)) to
            # (len(threshold_candidates), num_samples).
            # This transposed array of true labels now has the same shape as the preds array
            # (which has predictions for the current label across all threshold candidates).
            # This allows for a direct element-wise comparison between the true labels and the predictions
            # for each threshold when calculating metrics like precision, recall, and F1-score
            # using precision_recall_fscore_support.

            # average=None: metrics should be calculated for each class separately, rather than averaging them
            # zero_division='warn': how to handle division by zero
            # support (numpy.ndarray of ints) = number of true occurrences of each class

            # y_true : array-like (list, NumPy array, torch tensor converted to NumPy) <class 'numpy.ndarray'> shape=(5000,) = 1D array of true binary values (0 or 1) for a single label across all samples
            # y_pred : array-like (list, NumPy array, torch tensor converted to Numpy) <class 'numpy.ndarray'> shape=(5000,) = 1D array of predicted binary values (0 or 1) for the same label across all samples
            # labels : irrelevant in binary mode (1 label at a time)
            precision, recall, f1, supp = precision_recall_fscore_support(
                np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1)).T, preds, average=None, zero_division='warn'
            )
            if label_idx < 10:
                o1 = true_labels[:, label_idx]
                logger.debug(f"predict_with_optimized_thresholds: {label_idx}: true_labels[:, label_idx]: {type(o1)} shape={o1.shape}\n{o1}")
                # <class 'numpy.ndarray'> shape=(5000,)
                # [0. 1. 0. 0. 0. 0. 0. etc.]
                o2 = np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1))
                logger.debug(f"predict_with_optimized_thresholds: {label_idx}: np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1)): {type(o2)} shape={o2.shape}\n{o2}")
                # <class 'numpy.ndarray'> shape=(19, 5000)
                # [[0. 1. 0. 0. 0. 0. 0. etc.]
                # [0. 1. 0. 0. 0. 0. 0. etc.]
                # etc.
                # [0. 1. 0. 0. 0. 0. 0. etc.]]
                o3 = np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1)).T
                logger.debug(f"predict_with_optimized_thresholds: o3: {label_idx}: {type(o3)} shape={o3.shape}\n{o3}")
                # <class 'numpy.ndarray'> shape=(5000, 19)
                # [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
                #  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
                # etc.
                #  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

                logger.debug(f"predict_with_optimized_thresholds: precision: {type(precision)} shape={precision.shape}\n{precision}")
                # <class 'numpy.ndarray'> shape=(19,)
                # [0.14285714 0.14285714 etc.]
                logger.debug(f"predict_with_optimized_thresholds: recall   : {type(recall)}    shape={recall.shape}\n{recall}")
                # <class 'numpy.ndarray'> shape=(19,)
                # [1. 1. etc.]
                logger.debug(f"predict_with_optimized_thresholds: f1       : {type(f1)}        shape={f1.shape}\n{f1}")
                # <class 'numpy.ndarray'> shape=(19,)
                # [0.25 0.25 etc.]
                logger.debug(f"predict_with_optimized_thresholds: support  : {type(supp)}      shape={supp.shape}\n{supp}")
                # <class 'numpy.ndarray'> shape=(19,)
                # [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

            # Find the best threshold based on F1 for the current label
            best_idx                   = np.argmax(f1)
            best_thresholds[label_idx] = threshold_candidates[best_idx]
            # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> threshold_floor min <<<<<<<<<<
            best_thresholds[label_idx]   = np.maximum(best_thresholds[label_idx], threshold_floor)
            best_metrics[int(label_str)] = {'f1': f1[best_idx], 'precision': precision[best_idx], 'recall': recall[best_idx], 'support': supp[best_idx]}

            logger.debug(f"predict_with_optimized_thresholds: best_idx       : {type(best_idx)} {best_idx}")
            # <class 'numpy.int64'> 0
            logger.debug(f"predict_with_optimized_thresholds: best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
            # <class 'numpy.ndarray'> shape=(147,)
            # [0.05 0., [0.05 0.05 0. etc. ]
            logger.debug(f"predict_with_optimized_thresholds: best_metrics   : {type(best_metrics)} len={len(best_metrics)}\n{best_metrics}")
            # <class 'dict'> len=147

        best_thresholds_dict = {int(id2label[i]): best_thresholds[i].item() for i in range(len(best_thresholds))}
        logger.debug(f"predict_with_optimized_thresholds: best_thresholds_dict: {type(best_thresholds_dict)} len={len(best_thresholds_dict)}\n{best_thresholds_dict}")
        # <class 'dict'> len=147
        # {142: 0.49999999999999994, 143: 0.2, 146: 0.39999999999999997, etc.}
        # LABELS ARE int, ORDER IS bySkillTypeIdComplexityId <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        # Generate predictions using the optimized threshold for each label
        # probs          : <class 'numpy.ndarray'> shape=(5000, 147)
        # best_thresholds: <class 'numpy.ndarray'> shape=(147,)

        #best_predsZZZ = np.array([(probs[:, i] > best_thresholds[i]).astype(int) for i in range(num_labels)]).T
        #o1 = np.array([(probs[:, i] > best_thresholds[i]).astype(int) for i in range(num_labels)])
        #logger.debug(f"predict_with_optimized_thresholds: o1: {type(o1)} shape={o1.shape}\n{o1}")
        # probs[:, i]: all 5000 samples for the i-th labels => shape=(5000, )
        # best_thresholds[i]: scalar
        # (probs[:, i] > best_thresholds[i]).astype(int) => shape=(5000, )
        # repeat this for 147 labels => <class 'numpy.ndarray'> shape=(147, 5000)
        #logger.debug(f"predict_with_optimized_thresholds: best_predsZZZ: {type(best_predsZZZ)} shape={best_predsZZZ.shape}\n{best_predsZZZ}")
        # <class 'numpy.ndarray'> shape=(5000, 147)

        best_preds = (probs > best_thresholds).astype(int)
        logger.debug(f"predict_with_optimized_thresholds: best_preds: {type(best_preds)} shape={best_preds.shape}\n{best_preds}")
        # <class 'numpy.ndarray'> shape=(5000, 147)
    else:
        # Apply provided thresholds or default to 0.5
        thresholds_fixed = thresholds if thresholds is not None else np.full(num_labels, 0.5)
        logger.debug(f"predict_with_optimized_thresholds: thresholds_fixed: {type(thresholds_fixed)} shape={thresholds_fixed.shape}\n{thresholds_fixed}")
        # <class 'numpy.ndarray'> shape=(147,)

        # Compute predictions with fixed thresholds
        preds_fixed = (probs > thresholds_fixed).astype(int)
        logger.debug(f"predict_with_optimized_thresholds: preds_fixed: {type(preds_fixed)} shape={preds_fixed.shape}\n{preds_fixed}")
        # <class 'numpy.ndarray'> shape=(5000, 147)

        precision, recall, f1, supp = precision_recall_fscore_support(true_labels, preds_fixed, average=None, zero_division='warn')
        logger.debug(f"predict_with_optimized_thresholds: precision: {type(precision)} shape={precision.shape}\n{precision}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: recall   : {type(recall)}    shape={recall.shape}\n{recall}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: f1       : {type(f1)}        shape={f1.shape}\n{f1}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: support  : {type(supp)}      shape={supp.shape}\n{supp}")
        # <class 'numpy.ndarray'> shape=(147,)

        # Convert to dict format
        metrics_fixed         = {int(id2label[i]): {'f1': f1[i], 'precision': precision[i], 'recall': recall[i]} for i in range(num_labels)}
        thresholds_fixed_dict = {int(id2label[i]): thresholds_fixed[i].item() for i in range(num_labels)}

    thresholds      = best_thresholds      if threshold_tuning else thresholds_fixed
    thresholds_dict = best_thresholds_dict if threshold_tuning else thresholds_fixed_dict
    metrics         = best_metrics         if threshold_tuning else metrics_fixed
    preds           = best_preds           if threshold_tuning else preds_fixed


    # micro average
    #   compute metrics globally by summing all TP, FP, FN across all labels
    #   good for overall performance assessment
    #   dominated by frequent labels: if most samples belong to a few labels, it favors them

    # macro average
    #   each label is treated equally, regardless of how often it appears (= support)
    #   good for evaluating rare labels
    #   sensitive to rare labels: if rare labels perform poorly, macro F1 will drop

    # weighted average
    #   like macro, but weights each label by support
    #   balances between micro and macro by considering both label importance and prevalence
    #   useful if class imbalance exists but you still want per-label influence

    # samples average
    #  Computes metrics per sample, then averages. Best for multi-label classification.

    # support: number of true instances for each label

    global_metrics = {}
    for avg in ['micro', 'macro', 'weighted', 'samples']:
        p, r, f, s = precision_recall_fscore_support(true_labels, preds, average=avg, zero_division='warn')
        global_metrics[avg] = {'f1': f, 'precision': p, 'recall': r, 'support': s}

    return thresholds, thresholds_dict, metrics, global_metrics


In [ ]:
def predict_with_optimized_thresholds(
        trainer,
        dataset,
        id2label,  # {0: '142', 1: '143', 2: '146', 3: '147', 4: '149', 5: '150', etc.
        threshold_tuning = False,
        thresholds       = None
    ):
    """
    Predicts using trainer.predict(), with optional threshold tuning, using NumPy arrays, and not PyTorch tensors.

    Parameters:
    - trainer                         : Hugging Face Trainer or CustomTrainer instance
    - dataset                         : Dataset to predict on
    - id2label                        : Dictionary mapping label indices (int) to label names (str)
    - threshold_tuning                : Boolean to enable threshold tuning per class (i.e. per label)
    - thresholds       (numpy.ndarray): Custom thresholds for classification

    Returns:
    if threshold_tuning:
        - best_thresholds      (numpy.ndarray): optimized threshold per class (label)
        - best_thresholds_dict (dict)         : optimized threshold per class (label)
        - best_metrics         (dict)         : best F1, best precision, best recall per class (label)
        - best_preds           (numpy.ndarray): best predictions per class (label)
    else:
        - thresholds      (numpy.ndarray): fixed threshold per class (label)
        - thresholds_dict (dict)         : fixed threshold per class (label)
        - metrics         (dict)         : computed with provided thresholds or default to 0.5
        - preds           (numpy.ndarray): predictions with provided thresholds or default to 0.5
    """
    # Predict
    predictions_output = trainer.predict(dataset)
    logits             = predictions_output.predictions # raw predictions made by the model
    true_labels        = predictions_output.label_ids   # ground truth labels
    metrics            = predictions_output.metrics     # metrics computed during evaluation (NOT USED)

    # Sigmoid function: convert logits to probabilities (with np, not with torch)
    probs = 1 / (1 + np.exp(-logits))

    logger.debug(f"predict_with_optimized_thresholds: id2label          : {type(id2label)} len={len(id2label)}\n{id2label}")
    # <class 'dict'> len=147 {0: '142', 1: '143', etc. : order of labels = by SkillTypeIdComplexityId
    logger.debug(f"predict_with_optimized_thresholds: predictions_output: {type(predictions_output)} len={len(predictions_output)}")
    # <class 'transformers.trainer_utils.PredictionOutput'> len=3: tuple (predictions_output.predictions / label_ids /metrics)
    logger.debug(f"predict_with_optimized_thresholds: logits            : {type(logits)} shape={logits.shape}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: true_labels       : {type(true_labels)} shape={true_labels.shape}\n{true_labels}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: probs             : {type(probs)} shape={probs.shape}")
    # class <'numpy.ndarray'> shape=(5000, 147)
    logger.debug(f"predict_with_optimized_thresholds: metrics           : {type(metrics)} len={len(metrics)}\n{metrics}")
    # <class 'dict'> len=12
    # {'test_loss': 0.2430032342672348, 'test_f1': 0.11612903225806452, 'test_precision': 0.06338028169014084, 'test_recall': 0.6923076923076923, 'test_roc_auc': 0.5492063492063493, 'test_precision_recall_auc': 0.4570294784580498, 'test_subset_accuracy': 0.0, 'test_hamming_loss': 0.48324514991181655, 'test_hamming_accuracy': 0.5167548500881834, 'test_runtime': 0.1912, 'test_samples_per_second': 36.62, 'test_steps_per_second': 5.231}

    num_labels       = probs.shape[1]  # = number of columns = number of labels = len(id2label) = 147
    best_thresholds  = best_thresholds_dict  = best_metrics  = best_preds  = None
    thresholds_fixed = thresholds_fixed_dict = metrics_fixed = preds_fixed = None

    if threshold_tuning:
        best_thresholds, best_score = optimize_thresholds_for_exact_match(probs, true_labels, bounds=None, seed=42)
        # (num_labels,) array of thresholds optimized for exact match
        best_thresholds_dict        = {id2label[i]: best_thresholds[i].item() for i in range(len(best_thresholds))}
        best_preds                  = (probs > best_thresholds).astype(int)
        precision, recall, f1, supp = precision_recall_fscore_support(true_labels, best_preds, average=None, zero_division='warn')
        # (precision, recall, f1, supp)
        best_metrics                = {id2label[i]: {'f1': f1[i], 'precision': precision[i], 'recall': recall[i], 'support': supp[i]} for i in range(num_labels)}
    else:
        # Apply provided thresholds or default to 0.5
        thresholds_fixed = thresholds if thresholds is not None else np.full(num_labels, 0.5)
        logger.debug(f"predict_with_optimized_thresholds: thresholds_fixed: {type(thresholds_fixed)} shape={thresholds_fixed.shape}\n{thresholds_fixed}")
        # <class 'numpy.ndarray'> shape=(147,)

        # Compute predictions with fixed thresholds
        preds_fixed = (probs > thresholds_fixed).astype(int)
        logger.debug(f"predict_with_optimized_thresholds: preds_fixed: {type(preds_fixed)} shape={preds_fixed.shape}\n{preds_fixed}")
        # <class 'numpy.ndarray'> shape=(5000, 147)

        precision, recall, f1, supp = precision_recall_fscore_support(true_labels, preds_fixed, average=None, zero_division='warn')
        logger.debug(f"predict_with_optimized_thresholds: precision: {type(precision)} shape={precision.shape}\n{precision}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: recall   : {type(recall)}    shape={recall.shape}\n{recall}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: f1       : {type(f1)}        shape={f1.shape}\n{f1}")
        # <class 'numpy.ndarray'> shape=(147,)
        logger.debug(f"predict_with_optimized_thresholds: support  : {type(supp)}      shape={supp.shape}\n{supp}")
        # <class 'numpy.ndarray'> shape=(147,)

        # Convert to dict format
        metrics_fixed         = {int(id2label[i]): {'f1': f1[i], 'precision': precision[i], 'recall': recall[i]} for i in range(num_labels)}
        thresholds_fixed_dict = {int(id2label[i]): thresholds_fixed[i].item() for i in range(num_labels)}

    thresholds      = best_thresholds      if threshold_tuning else thresholds_fixed
    thresholds_dict = best_thresholds_dict if threshold_tuning else thresholds_fixed_dict
    metrics         = best_metrics         if threshold_tuning else metrics_fixed
    preds           = best_preds           if threshold_tuning else preds_fixed


    # micro average
    #   compute metrics globally by summing all TP, FP, FN across all labels
    #   good for overall performance assessment
    #   dominated by frequent labels: if most samples belong to a few labels, it favors them

    # macro average
    #   each label is treated equally, regardless of how often it appears (= support)
    #   good for evaluating rare labels
    #   sensitive to rare labels: if rare labels perform poorly, macro F1 will drop

    # weighted average
    #   like macro, but weights each label by support
    #   balances between micro and macro by considering both label importance and prevalence
    #   useful if class imbalance exists but you still want per-label influence

    # samples average
    #  Computes metrics per sample, then averages. Best for multi-label classification.

    # support: number of true instances for each label

    global_metrics = {}
    for avg in ['micro', 'macro', 'weighted', 'samples']:
        p, r, f, s = precision_recall_fscore_support(true_labels, preds, average=avg, zero_division='warn')
        global_metrics[avg] = {'f1': f, 'precision': p, 'recall': r, 'support': s}

    return thresholds, thresholds_dict, metrics, global_metrics


#### Evaluation 2: calculate metrics and optimized thresholds

- First, to **calculate** the optimized thresholds, threshold_tuning = True and thresholds = None.
- After, to **use** these optimized thresholds, threshold_tuning = False and thresholds = the optimized thresholds.

In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

# with best_thresholds (threshold_tuning = True and thresholds = None)
optimized_thresholds, optimized_thresholds_dict, evaluation_trainer_predict_metrics, evaluation_trainer_predict_global_metrics = predict_with_optimized_thresholds(
    trainer, validation_dataset, id2label, threshold_tuning=True, thresholds=None)
optimized_thresholds_native                      = convert_to_native(optimized_thresholds)
optimized_thresholds_dict_native                 = convert_to_native(optimized_thresholds_dict)
evaluation_trainer_predict_metrics_native        = convert_to_native(evaluation_trainer_predict_metrics)
evaluation_trainer_predict_global_metrics_native = convert_to_native(evaluation_trainer_predict_global_metrics)

logger.debug(f"optimized_thresholds: {optimized_thresholds}")
logger.debug(f"optimized_thresholds_dict: {optimized_thresholds_dict}")
logger.debug(f"evaluation_trainer_predict_metrics: {evaluation_trainer_predict_metrics}")
logger.debug(f"evaluation_trainer_predict_global_metrics: {evaluation_trainer_predict_global_metrics}")




logger.debug("==== with best thresholds ====")
logger.debug(f"optimized_thresholds_native                     : {type(optimized_thresholds_native)}                      len={len(optimized_thresholds_native)}                     \n{optimized_thresholds_native}")
# <class 'numpy.ndarray'> shape=(147,)
# [0.3  0.3 etc. 0.3 ]
logger.debug(f"optimized_thresholds_dict_native                : {type(optimized_thresholds_dict_native)}                 len={len(optimized_thresholds_dict_native)}                \n{optimized_thresholds_dict_native}")
# <class 'dict'> len=147
# {142: 0.3, 143: 0.3, etc.}
logger.debug(f"evaluation_trainer_predict_metrics_native       : {type(evaluation_trainer_predict_metrics_native)}        len={len(evaluation_trainer_predict_metrics_native)}       \n{evaluation_trainer_predict_metrics_native}")
# <class 'dict'> len=147
# {142: {'f1': np.float64(0.0), 'precision': np.float64(0.0), 'recall': np.float64(0.0), 'support': np.int64(0)}, 143: {'f1': np.float64(0.0), 'precision': np.float64(0.0), 'recall': np.float64(0.0), 'support': np.int64(0)},
logger.debug(f"evaluation_trainer_predict_global_metrics_native: {type(evaluation_trainer_predict_global_metrics_native)} len={len(evaluation_trainer_predict_global_metrics_native)}\n{evaluation_trainer_predict_global_metrics_native}")
# <class 'dict'> len=4
# {'micro': {'f1': 0.08798646362098139, 'precision': 0.04601769911504425, 'recall': 1.0, 'support': None}, 'macro': {
logger.debug("__________________________________________________________________")

logger.debug(f"after trainer.predict(): after best thresholds: multi_label_metrics_counter       : {multi_label_metrics_counter}")
logger.debug(f"after trainer.predict(): after best thresholds: compute_metrics_counter           : {compute_metrics_counter}")
logger.debug(f"after trainer.predict(): after best thresholds: compute_loss_counter              : {compute_loss_counter}")
logger.debug(f"after trainer.predict(): after best thresholds: CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
logger.debug(f"after trainer.predict(): after best thresholds: FocalLoss.counter                 : {FocalLoss.counter}")
logger.debug("__________________________________________________________________")

multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

# with thresholds_fixed 0.5 (threshold_tuning = False and thresholds = None)
thresholds_thr05, thresholds_dict_thr05, evaluation_trainer_predict_metrics_thr05, evaluation_trainer_predict_global_metrics_thr05 = predict_with_optimized_thresholds(
    trainer, validation_dataset, id2label, threshold_tuning=False, thresholds=None)
thresholds_thr05_native                                = convert_to_native(thresholds_thr05)
thresholds_dict_thr05_native                           = convert_to_native(thresholds_dict_thr05)
evaluation_trainer_predict_metrics_thr05_native        = convert_to_native(evaluation_trainer_predict_metrics_thr05)
evaluation_trainer_predict_global_metrics_thr05_native = convert_to_native(evaluation_trainer_predict_global_metrics_thr05)

logger.debug("==== with default fixed thresholds = 0.5 ====")
logger.debug(f"thresholds_thr05_native                               : {type(thresholds_thr05_native)}                                len={len(thresholds_thr05_native)}                               \n{thresholds_thr05_native}")
logger.debug(f"thresholds_dict_thr05_native                          : {type(thresholds_dict_thr05_native)}                           len={len(thresholds_dict_thr05_native)}                          \n{thresholds_dict_thr05_native}")
logger.debug(f"evaluation_trainer_predict_metrics_thr05_native       : {type(evaluation_trainer_predict_metrics_thr05_native)}        len={len(evaluation_trainer_predict_metrics_thr05_native)}       \n{evaluation_trainer_predict_metrics_thr05_native}")
logger.debug(f"evaluation_trainer_predict_global_metrics_thr05_native: {type(evaluation_trainer_predict_global_metrics_thr05_native)} len={len(evaluation_trainer_predict_global_metrics_thr05_native)}\n{evaluation_trainer_predict_global_metrics_thr05_native}")

logger.debug(f"after trainer.predict(): after 0.5 fixed threshold: multi_label_metrics_counter       : {multi_label_metrics_counter}")
logger.debug(f"after trainer.predict(): after 0.5 fixed threshold: compute_metrics_counter           : {compute_metrics_counter}")
logger.debug(f"after trainer.predict(): after 0.5 fixed threshold: compute_loss_counter              : {compute_loss_counter}")
logger.debug(f"after trainer.predict(): after 0.5 fixed threshold: CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
logger.debug(f"after trainer.predict(): after 0.5 fixed threshold: FocalLoss.counter                 : {FocalLoss.counter}")

logger.debug("__________________________________________________________________")

#raise Exception("QQQQ")


differential_evolution step 1: f(x)= -0.0
Polishing solution with 'L-BFGS-B'


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [ ]:
logger.info("evaluation 2: trainer.predict: successfully completed")


[INFO] evaluation 2: trainer.predict: successfully completed


#### Evaluation 2: trainer.predict: save locally and upload to HF Hub


In [ ]:
if ACCESS_TO_HF_HUB:
    name                            = "evaluation_trainer_predict_native"
    evaluation_trainer_predict_path = f"{name}.json"

    with open(evaluation_trainer_predict_path, "w") as f:
        json.dump(evaluation_trainer_predict_metrics_native, f)

    api.upload_file(
        path_or_fileobj = evaluation_trainer_predict_path,
        path_in_repo    = evaluation_trainer_predict_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_predict_path}")
    logger.debug("______________________________________________________________")

    name_thr05                            = "evaluation_trainer_predict_thr05_native"
    evaluation_trainer_predict_path_thr05 = f"{name_thr05}.json"

    with open(evaluation_trainer_predict_path_thr05, "w") as f:
        json.dump(evaluation_trainer_predict_metrics_thr05_native, f)

    api.upload_file(
        path_or_fileobj = evaluation_trainer_predict_path_thr05,
        path_in_repo    = evaluation_trainer_predict_path_thr05,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name_thr05}-{timestamp}"
    )

    logger.info(f"{name_thr05} successfully uploaded to HF Hub as {evaluation_trainer_predict_path_thr05}")

#raise Exception("RRRR")

[INFO] evaluation_trainer_predict_native successfully uploaded to HF Hub as evaluation_trainer_predict_native.json
[INFO] evaluation_trainer_predict_thr05_native successfully uploaded to HF Hub as evaluation_trainer_predict_thr05_native.json


#### Evaluation 2: optimized thresholds: save locally (as a dict) and upload to HF Hub (as a JSON file in repo 'dataset')
optimized_thresholds: <class 'numpy.ndarray'> shape=(6,) but JSON doesn't support NumPy types

In [ ]:
if THRESHOLD_TUNING and ACCESS_TO_HF_HUB:
    name                      = "optimized_thresholds_native"
    optimized_thresholds_path = f"{name}.json"

    with open(optimized_thresholds_path, "w") as f:
        json.dump(optimized_thresholds_dict_native, f)

    api.upload_file(
        path_or_fileobj = optimized_thresholds_path,
        path_in_repo    = f"{name}/{optimized_thresholds_path}",
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    # files.download(optimized_thresholds_path) is a command from the google.colab.files module that
    # initiates a download of the specified file from the Colab runtime to your local computer.
    # This is a convenient way to get files from the Colab environment to your machine.
    #files.download(optimized_thresholds_path)

    logger.info(f"{name} successfully uploaded to HF Hub as {optimized_thresholds_path}")


[INFO] optimized_thresholds_native successfully uploaded to HF Hub as optimized_thresholds_native.json


#### Evaluation 3: model.eval

In [ ]:
def compute_metrics_with_threshold(model, dataset, optimized_thresholds, id2label, batch_size=8):
    """
    Using a DataLoaderl, compute metrics during evaluation or test, by applying optimized thresholds

    Parameters:
    - model                                               : Hugging Face model
    - dataset                                             : Dataset to predict on
    - optimized_thresholds (list or NumPy array of floats): Optimized thresholds for each label
    - id2label                                            : Dictionary mapping label indices (int) to label names (string)
    - batch_size                                          : Batch size for prediction. Defaults to 8
    Returns:
    - metrics (dict)
    """
    dataloader = DataLoader(dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to GPU/CPU
    model.eval()      # Set model to evaluation mode

    all_logits, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            labels  = batch.pop('labels')                           # Keep 'labels' on CPU
            inputs  = {k: v.to(device) for k, v in batch.items()}   # Move inputs to device
            outputs = model(**inputs)

            logger.debug(f"compute_metrics_with_threshold: labels: {type(labels)}   shape={labels.shape}\n{labels}")
            # <class 'torch.Tensor'>   shape=torch.Size([5000, 147])
            # tensor([[0., 0., etc.], etc., [0., 0., etc., ]])
            logger.debug(f"compute_metrics_with_threshold: inputs: {type(inputs)}   shape={inputs['input_ids'].shape}\n{inputs['input_ids']}")
            # inputs is a dict of tensors:
            # - input_ids is a tensor on GPU with shape [5000, 1024]
            # - attention_mask: idem
            # <class 'dict'>   shape=torch.Size([5000, 1024])
            # tensor([[    0,  3750, etc.], etc., [    0, 28498, etc., 1, 1]], device='cuda:0')
            logger.debug(f"compute_metrics_with_threshold: outputs: {type(outputs)} len={len(outputs)}\n{outputs}")
            # <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> len=1
            # LongformerSequenceClassifierOutput(loss={'logits': tensor([[ 1.3489e-01, etc.], etc., ]], device='cuda:0')},
            # logits=tensor([[ 1.3489e-01, etc.]], device='cuda:0'), hidden_states=None, attentions=None, global_attentions=None)

            all_logits.append(outputs.logits.cpu())     # Keep logits as tensors, move to CPU
            all_labels.append(labels)                   # Labels remain on CPU

        logger.debug(f"compute_metrics_with_threshold: all_logits: {type(all_logits)} len={len(all_logits)}\n{all_logits}")
        # <class 'list'> len=1
        # [tensor([[ 1.3489e-01, etc.]])]
        logger.debug(f"compute_metrics_with_threshold: all_labels: {type(all_labels)} len={len(all_labels)}\n{all_labels}")
        # <class 'list'> len=1
        # [tensor([[0., 0., etc.]])]

    # Stack tensors
    logits = torch.cat(all_logits, dim=0).to(device)   # shape = (num_samples, num_labels), move to device
    labels = torch.cat(all_labels, dim=0).to(device)   # shape = (num_samples, num_labels), move to device
    logger.debug(f"compute_metrics_with_threshold: logits: {type(logits)} shape={logits.shape}\n{logits}")
    # <class 'torch.Tensor'> shape=torch.Size([5000, 147])
    # tensor([[ 1.3489e-01, etc.]], device='cuda:0')
    logger.debug(f"compute_metrics_with_threshold: labels: {type(labels)} shape={labels.shape}\n{labels}")
    # <class 'torch.Tensor'> shape=torch.Size([5000, 147])
    # tensor([[0., 0., etc.]], device='cuda:0')

    # Convert logits to probabilities
    probs = torch.sigmoid(logits)                      # shape = (num_samples, num_labels)
    logger.debug(f"compute_metrics_with_threshold: probs: {type(probs)} shape={probs.shape}\n{probs}")
    # <class 'torch.Tensor'> shape=torch.Size([5000, 147])
    # tensor([[0.5337, etc.]], device='cuda:0')

    # Apply per-class tuned thresholds (element-wise comparison)
    thresholds = torch.tensor(optimized_thresholds, dtype=torch.float32, device=device)  # Convert optimized_thresholds to tensor
    preds      = (probs > thresholds).int()                                              # Convert to binary predictions (1 or 0)
    logger.debug(f"compute_metrics_with_threshold: thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")
    # <class 'torch.Tensor'> shape=torch.Size([147])
    # ], device='cuda:0')
    logger.debug(f"compute_metrics_with_threshold: preds:      {type(preds)}      shape={preds.shape}     \n{preds}")
    # <class 'torch.Tensor'> shape=torch.Size([5000, 147])
    # tensor([[1, 1, etc.]], device='cuda:0', dtype=torch.int32)

    # Compute TP, FP, FN, TN
    TP = ((preds == 1) & (labels == 1)).sum(dim=0).float()
    TN = ((preds == 0) & (labels == 0)).sum(dim=0).float()
    FP = ((preds == 1) & (labels == 0)).sum(dim=0).float()
    FN = ((preds == 0) & (labels == 1)).sum(dim=0).float()
    logger.debug(f"compute_metrics_with_threshold: FN: {type(FN)} {FN}")  # <class 'torch.Tensor'> tensor([0., etc.], device='cuda:0')
    logger.debug(f"compute_metrics_with_threshold: TP: {type(TP)} {TP}")  # <class 'torch.Tensor'> tensor([0., etc.], device='cuda:0')
    logger.debug(f"compute_metrics_with_threshold: TN: {type(TN)} {TN}")  # <class 'torch.Tensor'> tensor([0., etc.], device='cuda:0')
    logger.debug(f"compute_metrics_with_threshold: FP: {type(FP)} {FP}")  # <class 'torch.Tensor'> tensor([7., etc.], device='cuda:0')

    # Compute per-class metrics
    precision_per_class = TP / (TP+FP+ 1e-8)
    recall_per_class    = TP / (TP+FN+ 1e-8)
    f1_per_class        = 2 * (precision_per_class*recall_per_class) / (precision_per_class+recall_per_class+1e-8)

    # Compute averaged metrics
    precision = precision_per_class.mean()
    recall    = recall_per_class.mean()
    f1        = f1_per_class.mean()
    logger.debug(f"compute_metrics_with_threshold: precision: {type(precision)} {precision}")  # <class 'torch.Tensor'> 0.045855384320020676
    logger.debug(f"compute_metrics_with_threshold: recall:    {type(recall)}    {recall}")     # <class 'torch.Tensor'> 0.25925925374031067
    logger.debug(f"compute_metrics_with_threshold: f1:        {type(f1)}        {f1}")         # <class 'torch.Tensor'> 0.07681755721569061

    # Compute accuracy
    accuracy = (preds == labels).float().mean()
    logger.debug(f"compute_metrics_with_threshold: accuracy: {type(accuracy)} {accuracy}")     # <class 'torch.Tensor'> 0.04585537686944008

    # Convert to NumPy for ROC-AUC and PR-AUC
    labels_np = labels.cpu().numpy()  # Move to CPU before converting
    probs_np  = probs.cpu().numpy()   # Move to CPU before converting
    logger.debug(f"compute_metrics_with_threshold: labels_np: {type(labels_np)} shape={labels_np.shape}\n{labels_np}")
    # <class 'numpy.ndarray'> shape=(5000, 147) [[0. 0. etc.]]
    logger.debug(f"compute_metrics_with_threshold: probs_np:  {type(probs_np)}  shape={probs_np.shape} \n{probs_np}")
    # <class 'numpy.ndarray'> shape=(5000, 147)  [[0.47941226 etc.]]

    # Compute ROC-AUC and PR-AUC
    roc_auc = torch.tensor(roc_auc_score(labels_np, probs_np, average=evaluation_average, multi_class='ovr'))
    pr_auc  = torch.tensor(average_precision_score(labels_np, probs_np, average=evaluation_average))
    logger.debug(f"compute_metrics_with_threshold: roc_auc: {type(roc_auc)} {roc_auc}")
    # <class 'torch.Tensor'> 0.4420615293985429
    logger.debug(f"compute_metrics_with_threshold: pr_auc:  {type(pr_auc)}  {pr_auc}")
    # class 'torch.Tensor'>  0.08582807126086

    # Convert predictions to Numpy for classification_report
    preds_np = preds.cpu().numpy()  # Move to CPU before converting
    logger.debug(f"compute_metrics_with_threshold: preds_np: {type(preds_np)} shape={preds_np.shape}\n{preds_np}")
    # <class 'numpy.ndarray'> shape=(5000, 147)  [[1 1 etc.]]

    # Generate classification report
    class_names  = [id2label[i] for i in range(len(id2label))]
    logger.debug(f"compute_metrics_with_threshold: id2label: {id2label}")
    # id2label: {0: '142', 1: '143', etc.}

    class_report = classification_report(labels_np, preds_np, target_names=class_names, zero_division='warn')
    #logger.debug(f"compute_metrics_with_threshold: Classification Report: {type(class_report)}\n{class_report}")
    # <class 'str'>
    #              precision    recall  f1-score   support

    # Store metrics
    metrics = {
        'accurary'             : accuracy.item(),
        'precision'            : precision.item(),
        'recall'               : recall.item(),
        'f1'                   : f1.item(),
        'roc_auc'              : roc_auc.item(),
        'pr_auc'               : pr_auc.item(),
        'per_class_precision'  : {id2label[i]: precision_per_class[i].item() for i in range(len(id2label))},
        'per_class_recall'     : {id2label[i]: recall_per_class[i].item()    for i in range(len(id2label))},
        'per_class_f1'         : {id2label[i]: f1_per_class[i].item()        for i in range(len(id2label))},
        'classification_report': class_report,
    }

    return metrics


In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

evaluation_model_eval_metrics        = compute_metrics_with_threshold(model, validation_dataset, optimized_thresholds, id2label, batch_size=16)
evaluation_model_eval_metrics_native = convert_to_native(evaluation_model_eval_metrics)

except_report = {k: v for k, v in evaluation_model_eval_metrics.items() if k!='classification_report'}
report        = evaluation_model_eval_metrics['classification_report']
logger.debug(f"evaluation_model_eval_metrics:                          {type(except_report)} len={len(except_report)}\n{except_report}")
logger.debug(f"evaluation_model_eval_metrics['classification_report']: {type(report)}        len={len(report)}       \n{report}")

logger.debug(f"after model.eval(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
logger.debug(f"after model.eval(): compute_metrics_counter           : {compute_metrics_counter}")
logger.debug(f"after model.eval(): compute_loss_counter              : {compute_loss_counter}")
logger.debug(f"after model.eval(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
logger.debug(f"after model.eval(): FocalLoss.counter                 : {FocalLoss.counter}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
logger.info("evaluation 3: model.eval: successfully completed")

[INFO] evaluation 3: model.eval: successfully completed


#### Evaluation 3: model.eval: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name                       = "evaluation_model_eval_native"
    evaluation_model_eval_path = f"{name}.json"

    with open(evaluation_model_eval_path, "w") as f:
        json.dump(evaluation_model_eval_metrics_native, f)

    api.upload_file(
        path_or_fileobj = evaluation_model_eval_path,
        path_in_repo    = evaluation_model_eval_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {evaluation_model_eval_path}")

#raise Exception("SSSS")


[INFO] evaluation_model_eval_native successfully uploaded to HF Hub as evaluation_model_eval_native.json


### Test PAS REVIEWED TEST

#### Test 1: trainer.evaluate

In [ ]:
test_trainer_evaluate_metrics = trainer.evaluate(
    eval_dataset = encoded_dataset['test'],
    metric_key_prefix='test'
)

logger.debug(f"test_trainer_evaluate_metrics: {type(test_trainer_evaluate_metrics)} len={len(test_trainer_evaluate_metrics)}\n{json.dumps(test_trainer_evaluate_metrics, indent=4)}")


In [ ]:
logger.info("test_trainer.evaluate successfully completed")


[INFO] test_trainer.evaluate successfully completed


#### Test 1: trainer.evaluate: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                       = "test_trainer_evaluate"
    test_trainer_evaluate_path = f"{name}.json"

    with open(test_trainer_evaluate_path, "w") as f:
        json.dump(test_trainer_evaluate_metrics, f)

    api.upload_file(
        path_or_fileobj = test_trainer_evaluate_path,
        path_in_repo    = test_trainer_evaluate_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} results successfully uploaded to HF Hub as {test_trainer_evaluate_path}")


[INFO] test_trainer_evaluate results successfully uploaded to HF Hub as test_trainer_evaluate.json


#### Test 2: trainer.predict

In [ ]:
optimized_thresholds, optimized_thresholds_dict, test_trainer_predict_metrics, test_trainer_predict_global_metrics = predict_with_optimized_thresholds(
    trainer, test_dataset, id2label, threshold_tuning=False, thresholds=optimized_thresholds)

logger.debug(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape} {optimized_thresholds}")
logger.debug(f"optimized_thresholds_dict: {type(optimized_thresholds_dict)} len={len(optimized_thresholds_dict)}\n{optimized_thresholds_dict}")
logger.debug(f"test_trainer_predict_metrics: {type(test_trainer_predict_metrics)} len={len(test_trainer_predict_metrics)}\n{json.dumps(test_trainer_predict_metrics, indent=4)}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

In [ ]:
logger.info("test_trainer.predict successfully completed")


[INFO] test_trainer.predict successfully completed


#### Test 2: trainer.predict: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                      = "test_trainer_predict"
    test_trainer_predict_path = f"{name}.json"

    with open(test_trainer_predict_path, "w") as f:
        json.dump(test_trainer_predict_metrics, f)

    api.upload_file(
        path_or_fileobj = test_trainer_predict_path,
        path_in_repo    = test_trainer_predict_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} results successfully uploaded to HF Hub as {test_trainer_predict_path}")


[INFO] test_trainer_predict results successfully uploaded to HF Hub as test_trainer_predict.json


#### Test 3: model.eval

In [ ]:
test_model_eval_metrics = compute_metrics_with_threshold(model, test_dataset, optimized_thresholds, id2label, batch_size=16)

except_report = {k: v for k, v in test_model_eval_metrics.items() if k!='classification_report'}
report        = test_model_eval_metrics['classification_report']
logger.debug(f"test_model_eval_metrics: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
logger.debug(f"test_model_eval_metrics['classification_report']: {type(report)} len={len(report)}\n{report}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Test 3: model.eval: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                 = "test_model_eval"
    test_model_eval_path = f"{name}.json"

    with open(test_model_eval_path, "w") as f:
        json.dump(test_model_eval_metrics, f)

    api.upload_file(
        path_or_fileobj = test_model_eval_path,
        path_in_repo    = test_model_eval_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    logger.info(f"{name} successfully uploaded to HF Hub as {test_model_eval_path}")


[INFO] test_model_eval successfully uploaded to HF Hub as test_model_eval.json


### Upload to HF Hub: blocking and no--blocking
- See Evernote: Hugging Face Hub: are push_to_hub and upload_file /upload_folder blocking?
- push_to_hub (for models/datasets): blocking by default
- upload_file / upload_folder: blocking by default
- => runtime.unassign() as last cell is safe

In [ ]:
logger.info("It's the end.")
download_logs()
time.sleep(5)


[INFO] It's the end.


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /content/logs/TrEvTe_colab_logs.zip   : 100%|##########|  516kB /  516kB            

# Only way to save the results table of trainer.train()
The output of trainer.train() (especially Hugging Face Trainer logs like the progress bar or result table) is often not stored in notebook output cells.

Instead, it is typically:

printed to the Colab terminal (stdout),

not captured as a proper cell output (like plots or print()),

thus not saved into the .ipynb file during download.

So your file looks correct in Colab because the output is displayed live during execution — but that terminal-style output (including the results table) is not saved in the file.

In [ ]:
#sys.exit(0)  # doesn't disconnect the runtime, doesn't stop consuming compute unit (Colab auto-disconnets after ~90 minutes)
#time.sleep(300)  # Wait 5 minutes before shutdown
#os.kill(os.getpid(), 9)
#runtime.unassign() # Colab no longer accept this!

==========================================================================================================

In [ ]:
# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

## Class supports, class weigths, weighted loss function

#Reminder:
#*   df_jobs      : <class 'pandas.core.frame.DataFrame'>
#*   df_jobs['id']: <class 'pandas.core.series.Series'>

#dataset = Dataset.from_pandas(df_jobs)
#*   dataset      : <class 'datasets.arrow_dataset.Dataset'>
#*   dataset['id']: <class 'list'>

#*   dataset_dict_jobs : <class 'datasets.dataset_dict.DatasetDict'>
#*   train_dataset     : <class 'datasets.arrow_dataset.Dataset'>
#*   validation_dataset: <class 'datasets.arrow_dataset.Dataset'>
#*   test_dataset      : <class 'datasets.arrow_dataset.Dataset'>


#We calculate the class supports for the train, validation and test datasets; the class weights and the weighted loss function are used for training only; the class supports of validation_dataset and test_dataset are calculated for information only.
# function B
def get_train_class_weights(datasetDict, labels):
  logger.debug(f"datasetDict: {type(datasetDict)} shape={datasetDict.shape}\n{datasetDict}")
  logger.debug(f"labels: {type(labels)} len={len(labels)}\n{labels}")

  dataset_train      = datasetDict['train']
  dataset_validation = datasetDict['validation']
  dataset_test       = datasetDict['test']

  def calculate_class_supports(dataset, labels):
    class_supports = dataset.map(
        lambda example: {col: example[col] for col in labels},
        batched=True
    ).to_pandas()[labels].sum(axis=0)
    return class_supports

  class_supports = {}

  for split_name, split_dataset in datasetDict.items():
    class_supports[split_name] = calculate_class_supports(split_dataset, labels)

  for split_name, split_class_supports in class_supports.items():
    logger.debug(f"{split_name}: {type(split_class_supports)} len={len(split_class_supports)}\n{split_class_supports}")

  train_class_supports_list = class_supports['train'].tolist()
  logger.debug(f"train_class_supports_list: {type(train_class_supports_list)} len={len(train_class_supports_list)} {train_class_supports_list}")

  train_class_supports_tensor = torch.tensor(train_class_supports_list, dtype=torch.float32)
  logger.debug(f"train_class_supports_tensor: {type(train_class_supports_tensor)} len={len(train_class_supports_tensor)} {train_class_supports_tensor}")

  train_total_samples = dataset_train.num_rows
  logger.debug(f"train_total_samples: {train_total_samples}")

  number_of_classes = len(labels)
  logger.debug(f"number_of_classes: {number_of_classes}")

  train_class_weights = train_total_samples / (number_of_classes * train_class_supports_tensor)
  logger.debug(f"train_class_weights: {type(train_class_weights)} len={len(train_class_weights)} {train_class_weights}")

  train_class_weights_sum = train_class_weights.sum()
  logger.debug(f"train_class_weights_sum: {train_class_weights_sum}")

  normalized_train_class_weights = (train_class_weights / train_class_weights_sum) * number_of_classes
  logger.debug(f"normalized_train_class_weights: {type(normalized_train_class_weights)} len={len(normalized_train_class_weights)} {normalized_train_class_weights}")

  # Positives samples per label
  supports = train_class_supports_tensor
  logger.debug(f"supports: {type(supports)} {len(supports)} {supports}")

  # Negatives samples per label
  negatives = train_total_samples - supports
  logger.debug(f"negatives: {type(negatives)} {len(negatives)} {negatives}")

  # pos_weights = negative to positive ratios
  pos_weights = negatives/supports
  logger.debug(f"pos_weights: {type(pos_weights)} {len(pos_weights)} {pos_weights}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  logger.debug(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  logger.debug(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  logger.debug(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  logger.debug(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using sum-to-one
  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  logger.debug(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  #return normalized_pos_weights_sum1

pos_weights = get_train_class_weights(datasetDict, labels)

loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))  # For multi-label classification (binary classification per label)
logger.debug(f"loss_fn: {type(loss_fn)} {loss_fn}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from your training results
epochs = np.arange(1, 9)
training_loss = [0.3122, 0.2897, 0.2665, 0.2466, 0.2244, 0.2223, 0.2202, 0.2077]
validation_loss = [0.303785, 0.293599, 0.278830, 0.275663, 0.280968, 0.280640, 0.279608, 0.282026]
f1_score = [0.865113, 0.871222, 0.875554, 0.880279, 0.879128, 0.878554, 0.879872, 0.877893]

# Plot Loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, label='Training Loss', marker='o')
plt.plot(epochs, validation_loss, label='Validation Loss', marker='s')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid()

# Plot F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_score, label='F1 Score', marker='o', color='green')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Score over Epochs')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
example = datasetDict['test'][0]
logger.debug(f"datasetDict['test'][0]: {type(example)} {example.keys()}\n{example}")

In [ ]:
inputs = tokenizer(
    example['text'],
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

In [ ]:
with torch.no_grad():  # Disable gradient calculation during prediction
    outputs = model(
        input_ids=inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device)
    )


In [ ]:
probs = torch.sigmoid(outputs.logits)


In [ ]:
best_thresholds = [0.4, 0.3, 0.3, 0.3, 0.4, 0.5]
preds = np.zeros_like(probs)  # Initialize predictions array
for label_idx in range(num_labels):
  preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx])  #.astype(int)

In [ ]:
logger.debug(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
logger.debug(f"preds: {type(preds)} shape={preds.shape}\n{preds}")

In [ ]:
logger.debug(f"labels: {type(labels)} {len(labels)}\n{labels}")
# '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}

In [ ]:
def tune_thresholds(true_labels, probs, id2label):
  """
  Tune thresholds for each label to maximize F1 alone, as F1 balances precision and recall into a single metric.

  Args:
    true_labels: actual labels for the data                                      (numpy array of shape (num_samples, num_labels))
    probs      : predicted probabilities                                         (numpy array of shape (num_samples, num_labels))
    id2label   : dictionary mapping label indices (int) to label names (string)

  Returns:
    best_thresholds: best threshold for each label                                                      (numpy array of shape (num_labels,))
    best_metrics   : dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label (dictionary of numpy arrays)
  """
  thresholds      = np.linspace(0.1, 0.9, 9)
  best_thresholds = np.zeros(len(id2label))
  best_metrics    = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

  for label_idx, label in id2label.items():
    for threshold in thresholds:
      pred                     = (probs[:, label_idx] > threshold).astype(int)
      precision, recall, f1, _ = precision_recall_fscore_support(true_labels[:, label_idx], pred, average='binary', zero_division=0)
      if f1 > best_metrics[label]['f1']:
        best_thresholds[label_idx]       = threshold
        best_metrics[label]['f1']        = f1
        best_metrics[label]['precision'] = precision
        best_metrics[label]['recall']    = recall

  logger.debug("==== tune_thresholds ====")
  logger.debug(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
  logger.debug(f"best_metrics   : {type(best_metrics)}    len={len(best_metrics)}      \n{json.dumps(best_metrics, indent=4)}")
  logger.debug("________________________________________________________________")

  return best_thresholds, best_metrics

In [ ]:
def compute_metrics_with_threshold_OLD(probs, label_ids, thresholds, id2label):
    """
    Compute metrics during evaluation or test, by applying tuned thresholds

    average:
    - 'micro'   : gives more weight to frequent labels     → best for imbalanced datasets where frequent labels are more important
    - 'macro'   : treats all labels equally                → best when you care about rare labels as much as frequent ones
    - 'weighted': like macro but considers label frequency → best if you want a compromise between macro and micro

    - 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
    - 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
    - 'weighted'   AUC: similar to macro but considers label frequency
    - 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
    - 'weighted PR AUC: also good, but slightly biased toward frequent labels

    PR AUC is better than ROC AUC when you care about positive examples in imbalanced data
    """
    average = 'macro'
    preds   = np.zeros_like(probs)

    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)

    # Compute metrics
    f1        = f1_score       (label_ids, preds, average=average)
    precision = precision_score(label_ids, preds, average=average)
    recall    = recall_score   (label_ids, preds, average=average)
    accuracy  = accuracy_score (label_ids, preds)

    # Compute AUC scores with error handling
    try:
        roc_auc              = roc_auc_score          (label_ids, probs, average=average)
    except ValueError:
        roc_auc              = 0.0

    try:
        precision_recall_auc = average_precision_score(label_ids, probs, average=average)
    except ValueError:
        precision_recall_auc = 0.0

    # Compute per-class metrics (average = None)
    per_class_f1        = f1_score       (label_ids, preds, average=None)
    per_class_precision = precision_score(label_ids, preds, average=None)
    per_class_recall    = recall_score   (label_ids, preds, average=None)

    # Generate classification report
    report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

    # Store metrics
    metrics = {
        'f1'                   : f1,
        'precision'            : precision,
        'recall'               : recall,
        'accuracy'             : accuracy,
        'roc_auc'              : roc_auc,
        'precision_recall_auc' : precision_recall_auc,
        'thresholds'           : thresholds.tolist(),
        'classification_report': report,
        'per_class_f1'         : per_class_f1,
        'per_class_precision'  : per_class_precision,
        'per_class_recall'     : per_class_recall
    }

    return metrics

In [ ]:
def compute_metrics_with_threshold_SAV(probs, label_ids, thresholds, id2label):

  #Compute metrics during evaluation or test, by applying tuned thresholds

  #logits  = eval_preds.predictions
  #labels  = eval_preds.label_ids
  #sigmoid = torch.nn.Sigmoid  # Sigmoid or numpy?
  #probs   = sigmoid(logits).cpu().numpy()
  preds   = np.zeros_like(probs)

  if THRESHOLD_TUNING:
    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)
  else:
    # threhsolds = None, apply a fixed threshold to all labels
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > threshold).astype(int)

  # Compute metrics
  f1                    = f1_score               (label_ids, preds, average='micro')
  precision             = precision_score        (label_ids, preds, average='micro')
  recall                = recall_score           (label_ids, preds, average='micro')
  accuracy              = accuracy_score         (label_ids, preds)
  roc_auc               = roc_auc_score          (label_ids, probs, average='micro')  # multi_class="ovr" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  precision_recall_auc  = average_precision_score(label_ids, probs, average='micro')

  # Use id2label for target_names
  report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

  if THRESHOLD_TUNING:
    _thresholds = thresholds.tolist()
  else:
    _thresholds = threshold

  metrics = {
      'f1'                   : f1,
      'precision'            : precision,
      'recall'               : recall,
      'accuracy'             : accuracy,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : _thresholds,
      'classification_report': report
  }

  return metrics

In [ ]:
def predict_with_optimized_thresholds_cuda(trainer, dataset, threshold_tuning=False, thresholds=None, threshold=threshold):
    """
    Predicts using trainer.predict(), with optional threshold tuning on GPU

    Parameters:
    - trainer         : Hugging Face Trainer or CustomTrainer instance
    - dataset         : dataset to predict on
    - id2label        : dictionary mapping label indices (int) to label names (string)
    - threshold_tuning: boolean to enable thresholds tuning per class (if evaluation, True, if prediction, False)
    - thresholds      : if evaluation, custom thresholds, if prediction, tuned thresholds (from evaluation)

    Returns:
    - best_thresholds (if threshold_tuning=True): optimized threshold for each label
    - best_metrics                              : computed with tuned thresholds whether for evaluation or prediction

    - metrics (if threshold_tuning=False): computed with fixed thresholds
    - metrics (if threshold_tuning=True): computed with tuned thresholds
    - predictions: final binary predictions
    - label_ids  : ground true labels from the dataset
    - best_thresholds (if threshold_tuning=True): optimized threshold per class
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Auto-detect GPU

    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions_np     = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (2400, 6)
    label_ids_np       = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (2400, 6)
    metrics_dict       = predictions_output.metrics      # <class 'dict'> (= trainer.evaluate results)           len   = 10

    logger.debug("==== predictions_output ====")
    logger.debug(f"predictions_output.predictions: {type(predictions_np)} shape={predictions_np.shape} \n{predictions_np}")
    logger.debug(f"predictions_output.label_ids  : {type(label_ids_np)}   shape={label_ids_np.shape}   \n{label_ids_np}")
    logger.debug(f"predictions_output.metrics    : {type(metrics_dict)}   len={len(metrics_dict)}      \n{json.dumps(metrics_dict, indent=4)}")

    # Convert NumPy arrays to PyTorch tensors (torch.from_numpy() keeps the NumPy array's memory layout, while torch.tensor() creates a new copy)
    logits      = torch.tensor(predictions_np, device=device)  # Move to GPU
    true_labels = torch.tensor(label_ids_np, device=device)    # Move to GPU

    logger.debug(f"logits     : {type(logits)}      shape={logits.shape}     \n{logits}")
    logger.debug(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")

    # Convert logits to probabilities using PyTorch (on GPU)
    probs = torch.sigmoid(logits)  # <class 'torch.Tensor'>  shape = (1200, 6)

    logger.debug(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
    logger.debug("__________________________________________________________________")
_
    num_labels = probs.shape[1]

    if THRESHOLD_TUNING:
        best_thresholds = torch.full((num_labels,), threshold, device=device, dtype=torch.float32)  # Default to threshold

        # Define candidate thresholds (on GPU)
        threshold_candidates = torch.linspace(0.05, 0.95, 19, device=device)

        for label_idx in range(num_labels):
            best_f1 = 0
            for threshold in threshold_candidates:
                preds = (probs[:, label_idx] > threshold).int()

                # Convert to CPU for sklearn
                precision, recall, f1, _ = precision_recall_fscore_support(
                    true_labels[:, label_idx].cpu().numpy(),
                    preds.cpu().numpy(),
                    average='binary',
                    zero_division=0
                )

                if f1 > best_f1:
                    best_f1                    = f1
                    best_thresholds[label_idx] = threshold  # Store best threshold

        return probs, true_labels, best_thresholds

    # Apply provided thresholds (or default to 0.5)
    if thresholds is None:
        thresholds = torch.full((num_labels,), 0.5, device=device)  # Default to 0.5
    else:
        thresholds = torch.tensor(thresholds, device=device)  # Move thresholds to GPU

    preds = (probs > thresholds).int()

    return preds, true_labels, None

In [ ]:
#test_trainer_predict_results = predict_with_optimized_thresholds(
#    trainer, test_dataset, id2label, threshold_tuning=False, thresholds=optimized_thresholds)

#except_report = {k: v for k, v in test_trainer_predict_results.items() if k!='classification_report'}
#report        = test_trainer_predict_results['classification_report']
#logger.debug(f"test_trainer_predict_results: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
#logger.debug(f"test_trainer_predict_results['classification_report']: {type(report)} len={len(report)}\n{report}")


In [ ]:
pos_weights = class_weights(train_dataset['labels'])
bce_loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))

